# OGGM冰川模拟结果分析与可视化

## 基于OGGM模型的气候极端事件对冰川质量平衡影响分析

**功能概述：**
- 读取OGGM冰川模拟结果（NetCDF格式）
- 与测高观测数据（ICESat/ICESat-2/CryoSat-2）进行对比验证
- 分析不同气候极端情景下的冰川响应
- 评估未来气候投影对冰川的影响
- 生成多种可视化图表

**数据来源：**
- OGGM模型输出（run_output_hist_hydro系列）
- ICESat/ICESat-2/CryoSat-2测高观测
- GRACE TWS数据
- CMIP6气候投影数据

**区域划分说明：**
- 1个HMA整体区域
- 15个GTN分区
- 22个HMA22子区域
- 1个TP（青藏高原）区域
- 496个网格单元

## 1. 导入必要的库和设置

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter
from matplotlib import cm
import matplotlib.colors as mcolors
import statsmodels.api as sm
from statsmodels.robust.robust_linear_model import RLM

from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes
from statsmodels.tsa.seasonal import STL
from datetime import datetime as dt
import time
import geopandas as gpd
import os
import warnings
import shapely.geometry as shpg
import xarray as xr
from hampel import hampel
from math import ceil, sqrt
from scipy import stats

from multiprocessing import Pool, cpu_count
from functools import partial
from tqdm.notebook import tqdm
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

warnings.filterwarnings("ignore")
sns.set(style="ticks")

# 设置中文字体和负号显示
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False

plt.rcParams['pdf.fonttype'] = 42 
plt.rcParams['ps.fonttype'] = 42 

print("✓ 库导入成功")

### 1.1 工具函数定义

In [ ]:
def toYearFraction(date):
    """将日期转换为年份小数形式"""
    def since_epoch(date):
        return time.mktime(date.timetuple())
    
    year = date.year
    start_of_this_year = dt(year=year, month=1, day=1)
    start_of_next_year = dt(year=year+1, month=1, day=1)
    
    year_elapsed = since_epoch(date) - since_epoch(start_of_this_year)
    year_duration = since_epoch(start_of_next_year) - since_epoch(start_of_this_year)
    fraction = year_elapsed / year_duration
    
    return date.year + fraction


def calc_nse(y_obs, y_sim):
    """计算Nash-Sutcliffe效率系数"""
    y_obs_mean = y_obs.mean()
    sum1 = ((y_obs - y_sim) ** 2).sum()
    sum2 = ((y_obs - y_obs_mean) ** 2).sum()
    nse = 1 - sum1 / sum2 if sum2 != 0 else np.nan
    return nse


def calc_r2(y_obs, y_sim):
    """计算相关系数（R²）"""
    y_obs_mean = y_obs.mean()
    y_sim_mean = y_sim.mean()
    
    numerator = ((y_sim - y_sim_mean) * (y_obs - y_obs_mean)).sum()
    denominator = np.sqrt(((y_sim - y_sim_mean) ** 2).sum() * ((y_obs - y_obs_mean) ** 2).sum())
    
    r2 = (numerator / denominator) ** 2 if denominator != 0 else np.nan
    return r2


def calc_rmse(y_obs, y_sim):
    """计算均方根误差"""
    rmse = np.sqrt(((y_sim - y_obs) ** 2).mean())
    return rmse


def calc_pbias(y_obs, y_sim):
    """计算百分比偏差"""
    sum1 = (y_sim - y_obs).sum()
    sum2 = y_obs.sum()
    pbias = 100 * (sum1 / sum2) if sum2 != 0 else np.nan
    return pbias


def get_file_path_list(path, filetype):
    """递归获取指定路径下所有指定类型的文件"""
    path_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(filetype):
                path_list.append(os.path.join(root, file))
    return path_list


def calculate_smb_from_volume(ds):
    """从体积数据计算比质量平衡率（SMB）
    
    参数:
        ds: xarray Dataset, 包含volume和area的OGGM输出
    返回:
        smb: numpy array, 比质量平衡率 (m w.e./yr)
    """
    smb = ((ds.volume.values[1:] - ds.volume.values[:-1]) / ds.area.values[1:]) * 0.9
    smb2 = ((ds.volume.values[1:] - ds.volume.values[:-1]) / ds.area.values[:-1]) * 0.9

    smb[np.isinf(smb)] = smb2[np.isinf(smb)]
    return smb


def calculate_elevation_change(smb):
    """从质量平衡计算累积高程变化
    Input:
        smb: N-1年质量平衡变化率， m w.e./yr
    Return:
        accumulated glacier elevation change for N-1 years, m, array
    
    """
    ele = np.cumsum(smb / 0.9)
    return ele


print("✓ 工具函数定义完成")

## 2. 数据路径配置

In [ ]:
# 配置数据路径（请根据实际情况修改）
CONFIG = {
    'oggm_dir': r'E:\revised_NCC_data\OGGM model\subregion_modeling\modeling_results_v2\[1]HMA',
    'icesat_path': r'E:\revised_NCC_data\ICESat_result\ICESat_seasonal_elevation_results.xlsx',
    'icesat2_path': r'E:\revised_NCC_data\ICESat2_result\ICESat2_seasonal_elevation_results.xlsx',
    'cryosat2_path': r'E:\revised_NCC_data\CryoSat2_result\CryoSat2_seasonal_elevation_results.xlsx',
    'grace_path': r'E:\HMA\GRACE_RL06\TWS_HMA_SSA.xlsx',
    'icesat_year_path': r'E:\revised_NCC_data\ICESat_result\ICESat_yearly_elevation_results.xlsx',
    'icesat2_year_path': r'E:\revised_NCC_data\ICESat2_result\ICESat2_yearly_elevation_results.xlsx',
    'cryosat2_year_path': r'E:\revised_NCC_data\CryoSat2_result\CryoSat2_yearly_elevation_results.xlsx',
    'altimetry_config_path': r'E:\HMA\测高结果20231109.xlsx',
    'hma_boundary': r'E:\HMA_subregion\regions_hma_v03_zheng\boundary_mountain_regions_hma_v3_zheng_20200601.shp',
    'tp_boundary': r'E:\HMA_subregion\TP_Boundary\TP_boundary.shp',
    'rgi_glacier': r'E:\HMA\Hugonnet\per_glacier\rgi_hma.shp',
    'output_dir': r'E:\revised_NCC_data\OGGM model\figure_plot'
}

# 获取OGGM数据目录列表
path = CONFIG['oggm_dir']
path_list = get_file_path_list(path, '.nc') if os.path.exists(path) else []
os.makedirs(CONFIG['output_dir'], exist_ok=True)

print(f"✓ 数据路径配置完成")
print(f"  找到 {len(path_list)} 个情景结果")

## 3. 加载观测数据和边界数据

### 3.1季节尺度数据

In [ ]:
# 加载测高观测数据
print("正在加载测高观测数据...")

# ICESat数据
icesat = pd.read_excel(CONFIG['icesat_path'], sheet_name='campaign_value', header=0, usecols='C:AQ')
icesat_uncert = pd.read_excel(CONFIG['icesat_path'], sheet_name='campaign_uncert', header=0, usecols='C:AQ')
icesat_num = pd.read_excel(CONFIG['icesat_path'], sheet_name='campaign_count', header=0, usecols='C:AQ')

# ICESat-2数据
icesat2 = pd.read_excel(CONFIG['icesat2_path'], sheet_name='month_value', header=0, usecols='A:AO')#0代表NaN
icesat2_uncert = pd.read_excel(CONFIG['icesat2_path'], sheet_name='month_uncert', header=0, usecols='A:AO')
icesat2_num = pd.read_excel(CONFIG['icesat2_path'], sheet_name='month_count', header=0, usecols='A:AO')

# CryoSat-2数据
cryosat2 = pd.read_excel(CONFIG['cryosat2_path'], sheet_name='month_value', header=0, usecols='A:AO')
cryosat2_uncert = pd.read_excel(CONFIG['cryosat2_path'], sheet_name='month_uncert', header=0, usecols='A:AO')
cryosat2_num = pd.read_excel(CONFIG['cryosat2_path'], sheet_name='month_count', header=0, usecols='A:AO')

# GRACE TWS数据
tws_trend = pd.read_excel(CONFIG['grace_path'], sheet_name='tws', header=499, nrows=255, usecols='D:AQ')
tws = pd.read_excel(CONFIG['grace_path'], sheet_name='tws', header=233, nrows=255, usecols='D:AQ')

# 测高结果（面积和校正值）
area = pd.read_excel(CONFIG['altimetry_config_path'], sheet_name='static_new', header=0, nrows=4, usecols='B:AN')
correction = pd.read_excel(CONFIG['altimetry_config_path'], sheet_name='static_new', header=49, nrows=4, usecols='B:AN')

head = area.columns.to_list()

# 初始化结果数据框
# rate = pd.DataFrame(data=np.zeros([2, len(head)]), columns=head)
# rate1 = pd.DataFrame(data=np.zeros([2, len(head)]), columns=head)
# rate2 = pd.DataFrame(data=np.zeros([2, len(head)]), columns=head)
# rate3 = pd.DataFrame(data=np.zeros([2, len(head)]), columns=head)

print("✓ 测高观测数据加载成功")
print(f"  区域数量: {len(head)}")

### 3.2 年尺度数据

In [ ]:
# ICESat数据路径
icesat_year_path = CONFIG['icesat_year_path']
icesat_year = pd.read_excel(
    icesat_year_path,
    sheet_name='year_value',
    header=0,
    usecols='A:AO'
)
icesat_uncert_year = pd.read_excel(
    icesat_year_path,
    sheet_name='year_uncert',
    header=0,
    usecols='A:AO'
)
icesat_num_year = pd.read_excel(
    icesat_year_path,
    sheet_name='year_count',
    header=0,
    usecols='A:AO'
)

# ICESat-2数据路径
icesat2_year_path = CONFIG['icesat2_year_path']
icesat2_year = pd.read_excel(
    icesat2_year_path,
    sheet_name='year_value',
    header=0,
    usecols='A:AO'
)
icesat2_uncert_year = pd.read_excel(
    icesat2_year_path,
    sheet_name='year_uncert',
    header=0,
    usecols='A:AO'
)
icesat2_num_year = pd.read_excel(
    icesat2_year_path,
    sheet_name='year_count',
    header=0,
    usecols='A:AO'
)

# CryoSat-2数据路径
cryosat2_year_path = CONFIG['cryosat2_year_path']
cryosat2_year = pd.read_excel(
    cryosat2_year_path,
    sheet_name='year_value',
    header=0,
    usecols='A:AO'
)
cryosat2_uncert_year = pd.read_excel(
    cryosat2_year_path,
    sheet_name='year_uncert',
    header=0,
    usecols='A:AO'
)
cryosat2_num_year = pd.read_excel(
    cryosat2_year_path,
    sheet_name='year_count',
    header=0,
    usecols='A:AO'
)

In [ ]:
# ========== 时间转换为年份小数 ==========
cryosat2 = cryosat2.rename(columns={'days':'mid_day'})
cryosat2_num = cryosat2_num.rename(columns={'days':'mid_day'})
cryosat2_uncert = cryosat2_uncert.rename(columns={'days':'mid_day'})

icesat2 = icesat2.rename(columns={'days':'mid_day'})
icesat2_num = icesat2_num.rename(columns={'days':'mid_day'})
icesat2_uncert = icesat2_uncert.rename(columns={'days':'mid_day'})

for idx, row in cryosat2.iterrows():
    cryosat2.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    cryosat2_num.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    cryosat2_uncert.loc[idx, 'mid_day'] = toYearFraction(row['date'])
cryosat2['mid_day'] = cryosat2['mid_day'].astype(float)
cryosat2_num['mid_day'] = cryosat2_num['mid_day'].astype(float)
cryosat2_uncert['mid_day'] = cryosat2_uncert['mid_day'].astype(float)

for idx, row in icesat2.iterrows():
    icesat2.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    icesat2_num.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    icesat2_uncert.loc[idx, 'mid_day'] = toYearFraction(row['date'])
icesat2['mid_day'] = icesat2['mid_day'].astype(float)
icesat2_num['mid_day'] = icesat2_num['mid_day'].astype(float)
icesat2_uncert['mid_day'] = icesat2_uncert['mid_day'].astype(float)

for idx, row in icesat.iterrows():
    icesat.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    icesat_num.loc[idx, 'mid_day'] = toYearFraction(row['date'])
    icesat_uncert.loc[idx, 'mid_day'] = toYearFraction(row['date'])
icesat['mid_day'] = icesat['mid_day'].astype(float)
icesat_num['mid_day'] = icesat_num['mid_day'].astype(float)
icesat_uncert['mid_day'] = icesat_uncert['mid_day'].astype(float)

# for year data
icesat_year_corr = icesat_year.iloc[:7, :]
icesat_year_corr[head] = icesat_year_corr[head] + correction.loc[2, :]
icesat_uncert_year = icesat_uncert_year.iloc[:7, :]
icesat_num_year = icesat_num_year.iloc[:7, :]

cryosat2_year_corr = cryosat2_year.iloc[:16, :]
cryosat2_year_corr[head] = cryosat2_year_corr[head] + correction.loc[0, :]
cryosat2_uncert_year = cryosat2_uncert_year.iloc[:16, :]
cryosat2_num_year = cryosat2_num_year.iloc[:16, :]

icesat2_year_corr = icesat2_year.iloc[:6, :]
icesat2_uncert_year = icesat2_uncert_year.iloc[:6, :]
icesat2_num_year = icesat2_num_year.iloc[:6, :]


### 3.3 加载空间边界数据

In [ ]:
# 加载空间边界数据
print("正在加载空间边界数据...")

HMA = gpd.read_file(CONFIG['hma_boundary'])
TP = gpd.read_file(CONFIG['tp_boundary'])
glacier_gdf = gpd.read_file(CONFIG['rgi_glacier'])

print("✓ 空间边界数据加载成功")
print(f"  HMA子区域数: {len(HMA)}")
print(f"  冰川数量: {len(glacier_gdf)}")

In [ ]:
# 需要保证坐标系一致
print(HMA.crs)
print(glacier_gdf.crs)

## 4. 数据预处理

### 4.1 裁剪到子区域

In [ ]:
import oggm_utils 
import importlib
importlib.reload(oggm_utils)
from oggm_utils import aggregate_to_region

worker_func = partial(aggregate_to_region, 
                    glacier_gdf=glacier_gdf, 
                    oggm_path_list=path_list, 
                    config=CONFIG)
NUM_CORES = 10
results = []

with Pool(processes=NUM_CORES) as pool:
    for res in tqdm(pool.imap_unordered(worker_func, HMA.iterrows()),
                                total=len(HMA), desc='处理区域'):
        results.append(res)

for res in results:
    print(res)

### 4.2 按冰川面积分组

In [ ]:
def group_by_area(glacier_gdf, area_min=None, area_max=None, output_prefix=''):
    """
    按冰川面积范围筛选并处理数据
    
    参数:
        area_min: float, 最小面积（km²）
        area_max: float, 最大面积（km²）
        output_prefix: str, 输出文件前缀
    """
    gdf_sel = glacier_gdf.copy()
    
    if area_min is not None:
        gdf_sel = gdf_sel.loc[gdf_sel['Area'] >= area_min]
    
    if area_max is not None:
        gdf_sel = gdf_sel.loc[gdf_sel['Area'] < area_max]
    
    print(f"筛选后冰川数量: {len(gdf_sel)}")
    
    files = get_file_path_list(CONFIG['oggm_dir'], '.nc')
    
    for i, file in enumerate(files):
        ds = xr.open_dataset(file)
        mask = ds['rgi_id'].isin(gdf_sel['RGIId'])
        ds_selected = ds.where(mask, drop=True)
        
        output_file = f"{CONFIG['output_path']}/categorized_by_area/{output_prefix}/{os.path.basename(file)}"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        ds_selected.to_netcdf(output_file)

        print(f"  已处理文件 {os.path.basename(file)}")
            


### 4.3 趋势计算

In [ ]:
# 计算特定时段趋势
def calculate_grid_trends(ele_grid, num_grid, suffix, start_time=None, end_time=None, return_hampel=True, use_num=True):
    """
    计算所有网格的ICESat-2高程变化趋势 (修改版)
    
    核心步骤：
    1. 应用数量阈值过滤（<20个观测值设为NaN）
    2. 使用Hampel滤波器去除时间序列中的异常值
    3. 鲁棒线性回归（RLM）计算趋势、p-value和95%置信区间
    
    Returns:
    --------
    pd.DataFrame
        包含每个网格趋势的DataFrame
        - 'Slope': 斜率/趋势 (m/yr)
        - 'Pvalue': p-value (显著性)
        - 'CI/2': 95%置信区间半宽度 (m/yr)
        - 'Intercept': 截距 (m)
    """
    head_grid = ele_grid.columns.tolist()[2:] # 1,2,...,496
    # 1. 应用数量阈值
    for i in head_grid:
        if use_num:
            mask = (num_grid[i] < 20) | (ele_grid[i] == 0) | (pd.isna(num_grid[i]))
        else:
            mask = pd.isna(num_grid[i])
        ele_grid.loc[mask, i] = np.nan
    
    # 初始化 results DataFrame
    results = pd.DataFrame(
        index=head_grid, 
        columns=['Slope', 'Pvalue', 'CI/2', 'Intercept'],
        dtype=float  # 预先指定数据类型
    )
    
    ele_grid_copy = ele_grid.copy()
    hampel_results = ele_grid.copy()
    hampel_results[head_grid] = np.nan
    if start_time is not None:
        ele_grid_copy = ele_grid_copy[(ele_grid_copy['date']>=start_time) & (ele_grid_copy['date']<end_time)]
    
    ele_grid_copy.sort_values(by='mid_day', inplace=True)
        
    for i in head_grid:
        valid_data = ele_grid_copy[[i, 'mid_day']].dropna()
        
        if len(valid_data) < 10:
            continue
        
        grid = valid_data.reset_index(drop=True)#连续索引
        year = grid['mid_day']
        dh = grid[i]
        
        # 2. Hampel滤波器去除异常值
        hamp_series = hampel(dh, window_size=5, n=3, imputation=True)
        hamp_series = hamp_series.astype(float)
        
        # 3. 鲁棒线性回归 (RLM)
        X = sm.add_constant(year)
        y = hamp_series
        
        rlm_model = RLM(y, X, M=sm.robust.norms.HuberT())
        rlm_results = rlm_model.fit()

        slope = rlm_results.params.iloc[1]
        intercept = rlm_results.params.iloc[0]
        pvalue = rlm_results.pvalues.iloc[1]
        
        # 计算95%置信区间
        conf_int = rlm_results.conf_int(alpha=0.05)
        # conf_int 是一个 DataFrame，[1] 是斜率的置信区间
        CI_half_width = (conf_int.iloc[1, 1] - conf_int.iloc[1, 0]) / 2

        results.loc[i, 'Slope'] = slope
        results.loc[i, 'Pvalue'] = pvalue
        results.loc[i, 'CI/2'] = CI_half_width
        results.loc[i, 'Intercept'] = intercept

    results.columns = [f'Slope_{suffix}', f'Pvalue_{suffix}', f'CI/2_{suffix}', f'Intercept_{suffix}']
    if not return_hampel:
        return results, None

    ele_grid2 = ele_grid.sort_values(by='mid_day', inplace=False)
    for i in head_grid:
        valid_data = ele_grid2[[i, 'mid_day']].dropna()
        if len(valid_data) < 10:
            continue

        grid = valid_data.reset_index(drop=True)#连续索引
        year = grid['mid_day']
        dh = grid[i]
        
        # 2. Hampel滤波器去除异常值
        hamp_series = hampel(dh, window_size=5, n=3, imputation=True)
        hamp_series = hamp_series.astype(float)
        hampel_results.loc[valid_data.index, i] = hamp_series.values
    
    
    return results, hampel_results

In [ ]:
icesat_corrected = icesat.copy()
cryosat2_corrected = cryosat2.copy()
tws_corrected = tws.copy()
tws_trend_corrected = tws_trend.copy()

for region_name in head:
    # ========== 系统偏差校正 ==========
    corr = correction.loc[0, region_name]   # CryoSat-2校正值
    corr2 = correction.loc[1, region_name]  # TWS校正值
    corr3 = correction.loc[2, region_name]  # ICESat校正值

    icesat_corrected[region_name] = icesat_corrected[region_name] + corr3
    cryosat2_corrected[region_name] = cryosat2_corrected[region_name] + corr
    #tws_corrected[region_name] = tws_corrected[region_name] + corr2
    #tws_trend_corrected[region_name] = tws_trend_corrected[region_name] + corr2

# ========== 计算趋势 ==========
trend03to09, hampel_icesat_corrected = calculate_grid_trends(icesat_corrected, 
                                                                icesat_num, 'ice1', 
                                                                start_time=None, end_time=None)#7 years
trend10to18, hampel_cryosat2_corrected = calculate_grid_trends(cryosat2_corrected, 
                                                                cryosat2_num, 'cryo2', 
                                                                start_time=dt(2010,1,1), end_time=dt(2019, 1, 1))#8.5 years
trend18to25, hampel_icesat2_corrected = calculate_grid_trends(icesat2, 
                                                                icesat2_num, 'ice2', 
                                                                start_time=dt(2018,1,1), end_time=dt(2025, 10, 1))#7 years
# 年结果在上面已经校正了

## 5. OGGM模拟结果与测高观测对比

### 5.1 提取并对比单个子区域的数据

In [ ]:
def compare_oggm_with_altimetry(region_idx, oggm_corr_list, output_dir=None, display_legend=False):
    """
    对比OGGM模拟结果与测高观测数据
    
    参数:
        region_idx: int, 区域索引（对应head列表）,16-37
        path_idx: int, OGGM数据路径索引
        oggm_corr_list: Dataframe
    """
    region_name = head[region_idx]
    region_name_clean = region_name

    if '.1' in region_name:
        region_name_clean = region_name_clean.replace('.1', '')
    if '/' in region_name:
        region_name_clean = region_name_clean.replace('/', '-')
        
    # 加载OGGM数据
    hist_filename = 'run_output_hist_spinup_all.nc'
    ds1 = xr.open_dataset(os.path.join(os.path.dirname(CONFIG['oggm_dir']), f'[{region_idx+1}]{region_name_clean}', hist_filename))\
                .sel(time=slice(2000, 2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True) 
    
    # 计算质量平衡和高程变化
    smb1 = calculate_smb_from_volume(ds1) # m w.s./yr, 2000-2024
    ele1 = calculate_elevation_change(smb1) # 累计高程变化，2000-2024，2024年整年
    
    # 绘制时间序列
    fig, ax = plt.subplots(figsize=(10, 6))

    scalor_factor = 4
    ax.plot(
        hampel_icesat_corrected['date'], hampel_icesat_corrected[region_name],
        '^-', color=plt.cm.Accent(0), linewidth=1, markersize=1.5,
        label='ICESat'
    )
    ax.plot(
        pd.to_datetime(icesat_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat_year_corr[region_name],
        '-', color=plt.cm.Accent(4), linewidth=2,
        label='ICESat annual mean'
    )
    ax.fill_between(
        pd.to_datetime(icesat_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat_year_corr[region_name] - icesat_uncert_year[region_name]*scalor_factor,
        icesat_year_corr[region_name] + icesat_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )

    ax.plot(
        hampel_cryosat2_corrected['date'], hampel_cryosat2_corrected[region_name],
        's-', color=plt.cm.Accent(1), linewidth=1, markersize=1.5,
        label='CryoSat-2'
    )
    ax.plot(
        pd.to_datetime(cryosat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        cryosat2_year_corr[region_name],
        '-', color=plt.cm.Accent(5), linewidth=2,
        label='CryoSat-2 annual mean'
    )
    ax.fill_between(
        pd.to_datetime(cryosat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        cryosat2_year_corr[region_name] - cryosat2_uncert_year[region_name]*scalor_factor,
        cryosat2_year_corr[region_name] + cryosat2_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )

    ax.plot(
        hampel_icesat2_corrected['date'], hampel_icesat2_corrected[region_name],
        'o-', color=plt.cm.Accent(2), linewidth=1, markersize=1.5,
        label='ICESat-2'
    )
    ax.plot(
        pd.to_datetime(icesat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), icesat2_year_corr[region_name],
        '-', color=plt.cm.Accent(6), linewidth=2,
        label='ICESat-2 annual mean'
    )
    ax.fill_between(
        pd.to_datetime(icesat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat2_year_corr[region_name] - icesat2_uncert_year[region_name]*scalor_factor,
        icesat2_year_corr[region_name] + icesat2_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )
    merged_year_series = pd.concat([icesat_year_corr.set_index('start_year')[region_name], #2003-2009
                                     cryosat2_year_corr.set_index('start_year').loc[2010:2018, region_name], #2010-2018
                                      icesat2_year_corr.set_index('start_year')[region_name]], axis=0) #2019-2024
    
    # 绘制OGGM模拟结果
    oggm_correction = oggm_corr_list.loc[region_name, 'OGGM_voffset']
    # timei = pd.date_range(start="07/01/2000", end="07/01/2024", freq='YS-JUL')
    timei = pd.date_range(start="01/01/2001", end="01/01/2025", freq='YS')
    ax.plot(timei, ele1 + oggm_correction, 'o-', color='k', 
            linewidth=2, markersize=3.5, label='OGGM')
    compare_year_corr = pd.DataFrame({'oggm_ele': ele1[3:] + oggm_correction,
                                        'obs_ele': merged_year_series.values}, index=np.arange(2003, 2025)) #2000-2025
    
    # 设置标题和标签
    title = region_name_clean
    ax.set_title(title, y=1.0, fontdict=dict(fontsize=28, color='black', 
                family='Arial', weight='bold'))
    # ax.set_ylabel('Elevation change / m', fontsize=13)
    ax.grid(axis='y', color='gray', linestyle='--', linewidth=0.3)
    ax.tick_params(length=4, width=1, labelsize=26)

    ax.text(0.1, 0.5, f'$CC$ = {compare_year_corr.corr(method='pearson')['oggm_ele']['obs_ele']:.2f}',
        fontsize=28, transform=ax.transAxes)

    # dh annual
    merge_corrected = pd.concat([icesat_year_corr.set_index('start_year')[region_name], #2003-2009
                        cryosat2_year_corr.set_index('start_year').loc[2010:2025, region_name]], axis=0) #2010-2024
    dh_annual = []
    # for year in range(2000, 2025):
    #     year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year-1, 11, 1)) & (merge_corrected['date']<dt(year, 4, 1)), region_name]
    #     next_year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year, 11, 1)) & (merge_corrected['date']<dt(year+1, 4, 1)), region_name]

    #     if year_row.empty or next_year_row.empty:
    #         dh_annual.append(np.nan)
    #     else:
    #         dh_annual.append(next_year_row.median() - year_row.median()) #2000， 20001整年
    for year in range(2004, 2025):
        year_row = merge_corrected.loc[merge_corrected.index==year-1] #2003/7
        next_year_row = merge_corrected.loc[merge_corrected.index==year] #2004/7

        if year_row.empty or next_year_row.empty:
            dh_annual.append(np.nan)
        else:
            dh_annual.append(next_year_row.values[0] - year_row.values[0]) #2000， 20001整年
    
    dh_merge = pd.DataFrame({'dh_annual': dh_annual},
                            index=np.arange(2004, 2025))
    dh_merge = dh_merge.dropna()
    ax_right = ax.twinx()
    bar_positions = pd.to_datetime(dh_merge.index.astype(int).astype(str) + '-07-01')
    ax_right.bar(bar_positions, dh_merge['dh_annual'], 
                width=300,  # 宽度为365天（约1年），可以根据需要调整
                color='skyblue',  # 蓝色
                edgecolor='none',  # 无边框
                alpha=0.5,
                zorder=0)  # 最底层
    ax_right.tick_params(length=4, width=1, labelsize=26)
    ax_right.set_ylim([int(dh_merge['dh_annual'].min())-1, 8])


    if display_legend:
        ax.legend(fontsize=24, ncol=4, 
           bbox_to_anchor=(2, 1.2, 2.5, 2),
           loc='lower right',
           mode='expand')
    
    # plt.tight_layout()
    # 保存结果
    if output_dir is not None:
        if display_legend:
            save_path = os.path.join(output_dir, f'【{region_idx+1}】{region_name_clean}_legend.pdf')
        else:
            save_path = os.path.join(output_dir, f'【{region_idx+1}】{region_name_clean}.pdf')
        os.makedirs(output_dir, exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()
    

In [ ]:
def compare_oggm_with_altimetry(region_idx, oggm_corr_list, output_dir=None, display_legend=False):
    """
    对比OGGM模拟结果与测高观测数据
    
    参数:
        region_idx: int, 区域索引（对应head列表）,16-37
        path_idx: int, OGGM数据路径索引
        oggm_corr_list: Dataframe
    """
    region_name = head[region_idx]
    region_name_clean = region_name

    if '.1' in region_name:
        region_name_clean = region_name_clean.replace('.1', '')
    if '/' in region_name:
        region_name_clean = region_name_clean.replace('/', '-')
        
    # 加载OGGM数据
    hist_filename = 'run_output_hist_spinup_all.nc'
    ds1 = xr.open_dataset(os.path.join(os.path.dirname(CONFIG['oggm_dir']), f'[{region_idx+1}]{region_name_clean}', hist_filename))\
                .sel(time=slice(2000, 2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True) 
    
    # 计算质量平衡和高程变化
    smb1 = calculate_smb_from_volume(ds1) # m w.s./yr, 2000-2024
    ele1 = calculate_elevation_change(smb1) # 累计高程变化，2000-2024，2024年整年
    
    # 绘制时间序列
    fig, ax = plt.subplots(figsize=(14, 6))

    scalor_factor = 4
    ax.plot(
        hampel_icesat_corrected['date'], hampel_icesat_corrected[region_name],
        '^-', color=plt.cm.Accent(0), linewidth=1, markersize=1.5,
        label='ICESat'
    )
    ax.plot(
        pd.to_datetime(icesat_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat_year_corr[region_name],
        '-', color=plt.cm.Accent(4), linewidth=2,
        label='ICESat annual mean'
    )
    ax.fill_between(
        pd.to_datetime(icesat_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat_year_corr[region_name] - icesat_uncert_year[region_name]*scalor_factor,
        icesat_year_corr[region_name] + icesat_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )

    ax.plot(
        hampel_cryosat2_corrected['date'], hampel_cryosat2_corrected[region_name],
        's-', color=plt.cm.Accent(1), linewidth=1, markersize=1.5,
        label='CryoSat-2'
    )
    ax.plot(
        pd.to_datetime(cryosat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        cryosat2_year_corr[region_name],
        '-', color=plt.cm.Accent(5), linewidth=2,
        label='CryoSat-2 annual mean'
    )
    ax.fill_between(
        pd.to_datetime(cryosat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        cryosat2_year_corr[region_name] - cryosat2_uncert_year[region_name]*scalor_factor,
        cryosat2_year_corr[region_name] + cryosat2_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )

    ax.plot(
        hampel_icesat2_corrected['date'], hampel_icesat2_corrected[region_name],
        'o-', color=plt.cm.Accent(2), linewidth=1, markersize=1.5,
        label='ICESat-2'
    )
    ax.plot(
        pd.to_datetime(icesat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), icesat2_year_corr[region_name],
        '-', color=plt.cm.Accent(6), linewidth=2,
        label='ICESat-2 annual mean'
    )
    ax.fill_between(
        pd.to_datetime(icesat2_year_corr['start_year'].astype(int).astype(str) + '-07-01'), 
        icesat2_year_corr[region_name] - icesat2_uncert_year[region_name]*scalor_factor,
        icesat2_year_corr[region_name] + icesat2_uncert_year[region_name]*scalor_factor,
        color='gray', alpha=0.3
    )
    merged_year_series = pd.concat([icesat_year_corr.set_index('start_year')[region_name], #2003-2009
                                     cryosat2_year_corr.set_index('start_year').loc[2010:2018, region_name], #2010-2018
                                      icesat2_year_corr.set_index('start_year')[region_name]], axis=0) #2019-2024
    
    # 绘制OGGM模拟结果
    oggm_correction = oggm_corr_list.loc[region_name, 'OGGM_voffset']
    # timei = pd.date_range(start="07/01/2000", end="07/01/2024", freq='YS-JUL')
    timei = pd.date_range(start="01/01/2001", end="01/01/2025", freq='YS')
    ax.plot(timei, ele1 + oggm_correction, 'o-', color='k', 
            linewidth=2, markersize=3.5, label='OGGM')
    compare_year_corr = pd.DataFrame({'oggm_ele': ele1[3:] + oggm_correction,
                                        'obs_ele': merged_year_series.values}, index=np.arange(2003, 2025)) #2000-2025
    
    # 设置标题和标签
    title = region_name_clean
    ax.set_title(title, y=1.0, fontdict=dict(fontsize=28, color='black', 
                family='Arial', weight='bold'))
    # ax.set_ylabel('Elevation change / m', fontsize=13)
    ax.grid(axis='y', color='gray', linestyle='--', linewidth=0.3)
    ax.tick_params(length=4, width=1, labelsize=26)

    ax.text(0.1, 0.5, f'$CC$ = {compare_year_corr.corr(method='pearson')['oggm_ele']['obs_ele']:.2f}',
        fontsize=28, transform=ax.transAxes)

    # dh annual
    merge_corrected = pd.concat([icesat_year_corr.set_index('start_year')[region_name], #2003-2009
                        cryosat2_year_corr.set_index('start_year').loc[2010:2025, region_name]], axis=0) #2010-2024
    dh_annual = []
    # for year in range(2000, 2025):
    #     year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year-1, 11, 1)) & (merge_corrected['date']<dt(year, 4, 1)), region_name]
    #     next_year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year, 11, 1)) & (merge_corrected['date']<dt(year+1, 4, 1)), region_name]

    #     if year_row.empty or next_year_row.empty:
    #         dh_annual.append(np.nan)
    #     else:
    #         dh_annual.append(next_year_row.median() - year_row.median()) #2000， 20001整年
    for year in range(2004, 2025):
        year_row = merge_corrected.loc[merge_corrected.index==year-1] #2003/7
        next_year_row = merge_corrected.loc[merge_corrected.index==year] #2004/7

        if year_row.empty or next_year_row.empty:
            dh_annual.append(np.nan)
        else:
            dh_annual.append(next_year_row.values[0] - year_row.values[0]) #2000， 20001整年
    
    dh_merge = pd.DataFrame({'dh_annual': dh_annual},
                            index=np.arange(2004, 2025))
    dh_merge = dh_merge.dropna()
    ax_right = ax.twinx()
    bar_positions = pd.to_datetime(dh_merge.index.astype(int).astype(str) + '-07-01')
    ax_right.bar(bar_positions, dh_merge['dh_annual'], 
                width=300,  # 宽度为365天（约1年），可以根据需要调整
                color='skyblue',  # 蓝色
                edgecolor='none',  # 无边框
                alpha=0.5,
                zorder=0)  # 最底层
    ax_right.tick_params(length=4, width=1, labelsize=26)
    ax_right.set_ylim([int(dh_merge['dh_annual'].min())-1, 8])


    if display_legend:
        ax.legend(fontsize=24, ncol=4, 
           bbox_to_anchor=(2, 1.2, 2.5, 2),
           loc='lower right',
           mode='expand')
    
    # plt.tight_layout()
    # 保存结果
    if output_dir is not None:
        if display_legend:
            save_path = os.path.join(output_dir, f'【{region_idx+1}】{region_name_clean}_legend.pdf')
        else:
            save_path = os.path.join(output_dir, f'【{region_idx+1}】{region_name_clean}.pdf')
        os.makedirs(output_dir, exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()
    

In [ ]:
oggm_corr_list = pd.DataFrame(
    {
        head[0]: -2.0,
        head[16]: -2.0,
        head[17]: -2.2,
        head[18]: -2.0,
        head[19]: -3.0,
        head[20]: -1.5,
        head[21]: -4.0,
        head[22]: -4.0,
        head[23]: -3.0,
        head[24]: -2.0,
        head[25]: -1.5,
        head[26]: -1.5,
        head[27]: -3.0,
        head[28]: -0.5,
        head[29]: -2.5,
        head[30]: -5.0,
        head[31]: -1.0,
        head[32]: -2.8,
        head[33]: -3.0,
        head[34]: -1.0,
        head[35]: -2.0,
        head[36]: -1.0,
        head[37]: -3.5,
    },
    index=[0]
).T

oggm_corr_list.columns = ['OGGM_voffset']

region_idx = 0
# compare_oggm_with_altimetry(region_idx, oggm_corr_list, 
#                             output_dir=None, #os.path.join(CONFIG['output_dir'], 'oggm_altimetry_compar', 'legend')
#                             display_legend=False)
compare_oggm_with_altimetry(region_idx, oggm_corr_list, 
                            output_dir=os.path.join(CONFIG['output_dir'], 'oggm_altimetry_compar', 'no_legend'),
                            display_legend=False)

In [ ]:
oggm_corr_list = pd.DataFrame(
    {
        head[0]: -2.0,
        head[16]: -2.0,
        head[17]: -2.2,
        head[18]: -2.0,
        head[19]: -3.0,
        head[20]: -1.5,
        head[21]: -4.0,
        head[22]: -4.0,
        head[23]: -3.0,
        head[24]: -2.0,
        head[25]: -1.5,
        head[26]: -1.5,
        head[27]: -3.0,
        head[28]: -0.5,
        head[29]: -2.5,
        head[30]: -5.0,
        head[31]: -1.0,
        head[32]: -2.8,
        head[33]: -3.0,
        head[34]: -1.0,
        head[35]: -2.0,
        head[36]: -1.0,
        head[37]: -3.5,
    },
    index=[0]
).T
oggm_corr_list.columns = ['OGGM_voffset']

region_indices = [0] + list(range(16,38))

for region_idx in region_indices:
    saved_path = os.path.join(CONFIG['output_dir'], 'oggm_altimetry_compar')

    if '.1' in saved_path:
        saved_path = saved_path.replace('.1', '')
    if '/' in saved_path:
        saved_path = saved_path.replace('/', '-')
    
    os.makedirs(os.path.dirname(saved_path), exist_ok=True)
    
    compare_oggm_with_altimetry(region_idx, oggm_corr_list, output_dir=saved_path, display_legend=False)

### 5.3 每年高程变化散点图

In [ ]:
# STL分解提取趋势项
icesat_trend_results = hampel_icesat_corrected[['mid_day', 'date']].copy()
cry_trend_results = hampel_cryosat2_corrected[['mid_day', 'date']].copy()

for region in head:
    region_data = hampel_icesat_corrected[['date', region]].dropna()
    stl = STL(
            region_data[region].values, 
            period=3,
            robust=True
            )
    decomposition = stl.fit()
    trend_component = decomposition.trend
    
    icesat_trend_results[region] = np.nan
    icesat_trend_results.loc[region_data.index, region] = trend_component

    region_data = hampel_cryosat2_corrected[['date', region]].dropna()
    stl = STL(
            region_data[region].values, 
            period=12,
            robust=True,
            seasonal=13,
            )
    decomposition = stl.fit()
    trend_component = decomposition.trend

    cry_trend_results[region] = np.nan
    cry_trend_results.loc[region_data.index, region] = trend_component

merge_corrected = pd.concat([icesat_trend_results, cry_trend_results], axis=0)
merge_corrected = merge_corrected.sort_values('date').reset_index(drop=True)

In [ ]:
# 使用原始观测数据(保留季节项)
merge_corrected = pd.concat([hampel_icesat_corrected, hampel_cryosat2_corrected], axis=0)
merge_corrected = merge_corrected.sort_values('date').reset_index(drop=True)

In [ ]:
# 使用原始观测数据(保留季节项)
merge_corrected = pd.concat([hampel_icesat_corrected, 
                            hampel_cryosat2_corrected[hampel_cryosat2_corrected['date']<dt(2019,1,1)],
                            hampel_icesat2_corrected[hampel_icesat2_corrected['date']>=dt(2019,1,1)]], 
                            axis=0)
merge_corrected = merge_corrected.sort_values('date').reset_index(drop=True)

In [ ]:
def compare_scatter(region_idx, merge_corrected, output_dir=None, display_colorbar=False):
    """
    对比OGGM模拟结果与测高观测数据
    
    参数:
        region_idx: int, 区域索引（对应head列表）,16-37
        path_idx: int, OGGM数据路径索引
    """
    region_name = head[region_idx]
    region_name_clean = region_name

    if '.1' in region_name:
        region_name_clean = region_name_clean.replace('.1', '')
    if '/' in region_name:
        region_name_clean = region_name_clean.replace('/', '-')
        
    # 加载OGGM数据
    hist_filename = 'run_output_hist_spinup_all.nc'
    ds1 = xr.open_dataset(os.path.join(os.path.dirname(CONFIG['oggm_dir']), f'[{region_idx+1}]{region_name_clean}', hist_filename))\
                .sel(time=slice(2000, 2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True) 
    
    # 计算质量平衡和高程变化
    smb1 = calculate_smb_from_volume(ds1) # m w.s./yr, 2000-2024每年质量平衡
    dh_year = smb1 / 0.9 # 每年冰川高程变化，2000年，2004年(准确来说，2001.1、2002.1、2003.1)

    # 计算整年的测高高程变化，from 2000 to 2024
    altimetry_result = []
    for year in range(2000, 2025):
        year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year-1, 11, 1)) & (merge_corrected['date']<dt(year, 4, 1)), region_name]
        next_year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year, 11, 1)) & (merge_corrected['date']<dt(year+1, 4, 1)), region_name]

        if year_row.empty or next_year_row.empty:
            altimetry_result.append(np.nan)
        else:
            altimetry_result.append(next_year_row.median() - year_row.median()) #2000， 20001整年
    
    dh_merge = pd.DataFrame({'altimetry': altimetry_result, 'oggm': dh_year},
                            index=np.arange(2000, 2025))
    dh_merge = dh_merge.dropna()
    # 绘制时间序列
    fig, ax = plt.subplots(figsize=(6, 6))

    # 散点图
    scatter = ax.scatter(dh_merge['altimetry'], dh_merge['oggm'], s=120, c=dh_merge.index,
                vmin=2000, vmax=2024, cmap='Reds', 
                edgecolors='none', marker='o', alpha=0.7)
    x_lim = ax.get_xlim() 
    y_lim = ax.get_ylim()
    axis_min = min(x_lim[0], y_lim[0])
    axis_max = max(x_lim[1], y_lim[1])
    ax.plot([axis_min, axis_max], [axis_min, axis_max], linestyle='--', color='gray', linewidth=1.5, label='1:1 line')
    ax.set_xlim([axis_min, axis_max])
    ax.set_ylim([axis_min, axis_max])

    # 拟合
    x_data = dh_merge['altimetry'].values
    y_data = dh_merge['oggm'].values
    slope, intercept, _, _, _ = stats.linregress(x_data, y_data)
    x_fit = np.array([axis_min, axis_max])
    y_fit = slope * x_fit + intercept

    ax.plot(x_fit, y_fit, color='red', linewidth=2, linestyle='--')
    ax.axis('equal')
    # ax.vlines(0, axis_min, axis_max, linestyle='-', color='gray',linewidth=2)
    # ax.hlines(0, axis_min, axis_max, linestyle='-', color='gray',linewidth=2)

    R2 = calc_nse(dh_merge['altimetry'], dh_merge['oggm'])
    r = np.sqrt(calc_r2(dh_merge['altimetry'], dh_merge['oggm']))
    rmse = calc_rmse(dh_merge['altimetry'], dh_merge['oggm'])
    
    # 设置标题和标签
    title = region_name_clean
    ax.set_title(title, y=1.0, fontdict=dict(fontsize=28, color='black', 
                family='Arial', weight='bold'))
    # ax.set_ylabel('Elevation change / m', fontsize=13)
    ax.tick_params(length=7, width=3, labelsize=26)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    text = f'$r$: {r:.2f}\nRMSE: {rmse:.2f} m/yr'

    ax.text(0.05, 0.85, text,
        fontsize=24, transform=ax.transAxes, va='bottom')

    if display_colorbar:
        cbar_ax = fig.add_axes([0, -0.2, 2, 0.07]) 
        cbar = fig.colorbar(scatter, cax=cbar_ax, orientation='horizontal')
        cbar.set_label('Year', fontsize=26, labelpad=15)
        cbar.ax.tick_params(labelsize=24, length=7, width=3)
        cbar.set_ticks([2000, 2005, 2010, 2015, 2020, 2024])
    
    # plt.tight_layout()
    # 保存结果
    if output_dir is not None:
        save_path = os.path.join(output_dir, f'【{region_idx+1}】{region_name_clean}.pdf')
        os.makedirs(output_dir, exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()
    

In [ ]:
region_idx = 0
compare_scatter(region_idx, merge_corrected,
                            output_dir=r'E:\revised_NCC_data\OGGM model\figure_plot\oggm_altimetry_compar_eachyear\legend', 
                            display_colorbar=True)

In [ ]:
region_indices = [0] + list(range(16,38))

for region_idx in region_indices:
    saved_path = os.path.join(CONFIG['output_dir'], 'oggm_altimetry_compar_eachyear')
    
    os.makedirs(saved_path, exist_ok=True)
    
    compare_scatter(region_idx, merge_corrected,
                            output_dir=saved_path, 
                            display_colorbar=False)

In [ ]:
def compare_scatter(region_idlist, merge_corrected, output_dir=None, display_colorbar=False):
    """
    对比OGGM模拟结果与测高观测数据
    
    参数:
        region_idx: int, 区域索引（对应head列表）,16-37
        path_idx: int, OGGM数据路径索引
    """
    x_data = []
    y_data = []
    year_data = []
    for region_idx in region_idlist:
        region_name = head[region_idx]
        region_name_clean = region_name

        if '.1' in region_name:
            region_name_clean = region_name_clean.replace('.1', '')
        if '/' in region_name:
            region_name_clean = region_name_clean.replace('/', '-')
            
        # 加载OGGM数据
        hist_filename = 'run_output_hist_spinup_all.nc'
        ds1 = xr.open_dataset(os.path.join(os.path.dirname(CONFIG['oggm_dir']), f'[{region_idx+1}]{region_name_clean}', hist_filename))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True) 
        
        # 计算质量平衡和高程变化
        smb1 = calculate_smb_from_volume(ds1) # m w.s./yr, 2000-2024每年质量平衡
        dh_year = smb1 / 0.9 # 每年冰川高程变化，2000年，2004年

        # 计算整年的测高高程变化
        altimetry_result = []
        for year in range(2000, 2025):
            year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year-1, 7, 1)) & (merge_corrected['date']<dt(year, 7, 1)), region_name]
            next_year_row = merge_corrected.loc[(merge_corrected['date']>=dt(year, 7, 1)) & (merge_corrected['date']<dt(year+1, 7, 1)), region_name]

            if year_row.empty or next_year_row.empty:
                altimetry_result.append(np.nan)
            else:
                altimetry_result.append(next_year_row.median() - year_row.median()) #2000， 20001整年
        
        dh_merge = pd.DataFrame({'altimetry': altimetry_result, 'oggm': dh_year},
                                index=np.arange(2000, 2025))
        dh_merge = dh_merge.dropna()

        x_data.extend(dh_merge['altimetry'].values)
        y_data.extend(dh_merge['oggm'].values)
        year_data.extend(dh_merge.index.values)
    # 绘制时间序列
    fig, ax = plt.subplots(figsize=(12, 10))
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    year_data = np.array(year_data)

    # 散点图
    scatter = ax.scatter(x_data, y_data, s=100, c=year_data,
                vmin=2000, vmax=2024, cmap='Reds', 
                edgecolors='none', marker='o', alpha=0.7)
    x_lim = ax.get_xlim() 
    y_lim = ax.get_ylim()
    axis_min = min(x_lim[0], y_lim[0])
    axis_max = max(x_lim[1], y_lim[1])
    ax.plot([axis_min, axis_max], [axis_min, axis_max], linestyle='--', color='gray', linewidth=1.5, label='1:1 line')
    ax.set_xlim([axis_min, axis_max])
    ax.set_ylim([axis_min, axis_max])

    # 拟合
    X = sm.add_constant(x_data)
    rlm_model = RLM(y_data, X, M=sm.robust.norms.HuberT())
    rlm_results = rlm_model.fit()

    slope = rlm_results.params[1]
    intercept = rlm_results.params[0]
    y_pred = rlm_results.fittedvalues
    r_model = np.corrcoef(y_data, y_pred)[0, 1]
    x_fit = np.array([axis_min, axis_max])
    y_fit = slope * x_fit + intercept

    ax.plot(x_fit, y_fit, color='red', linewidth=2, linestyle='--')
    ax.axis('equal')

    # R2 = calc_nse(x_data, y_data)
    r = np.sqrt(calc_r2(x_data, y_data))
    rmse = calc_rmse(x_data, y_data)
    
    # 设置标题和标签
    # title = region_name_clean
    # ax.set_title(title, y=1.0, fontdict=dict(fontsize=28, color='black', 
    #             family='Arial', weight='bold'))
    # ax.set_ylabel('Elevation change / m', fontsize=13)
    ax.tick_params(length=7, width=3, labelsize=26)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    text = f'$r$: {r:.2f}\nRMSE: {rmse:.2f} m/yr'

    ax.text(0.05, 0.80, text,
        fontsize=26, transform=ax.transAxes, va='bottom')
    ax.set_xlabel('Annual elevation change from altimetry (m/yr)', fontsize=28)
    ax.set_ylabel('Annual elevation change from OGGM (m/yr)', fontsize=28)

    ax.set_xlim([-5, 4])
    ax.set_ylim([-5, 4])

    if display_colorbar:
        cbar = fig.colorbar(scatter, ax=ax, orientation='vertical')
        cbar.set_label('Year', fontsize=26, labelpad=15)
        cbar.ax.tick_params(labelsize=24, length=7, width=3)
        cbar.set_ticks([2000, 2005, 2010, 2015, 2020, 2024])
    
    # plt.tight_layout()
    # 保存结果
    if output_dir is not None:
        save_path = os.path.join(output_dir, f'HMA.pdf')
        os.makedirs(output_dir, exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()
    

In [ ]:
region_indices = [0] + list(range(16,38))
compare_scatter(region_indices, merge_corrected,
                            output_dir=r'E:\revised_NCC_data\OGGM model\figure_plot\oggm_altimetry_compar_eachyear\all_region_merge', 
                            display_colorbar=True)

## 6. 2023年气候敏感性分析

对比不同情景（原始、2023、2022-23、降水增强等）下的质量平衡与高程变化，并输出关键统计。


In [ ]:
def plot_sensitivity(region_dirs, save_dir, CONFIG):
    """
    分析2023年气候极端多情景下的质量平衡与高程变化对比。
    依赖：CONFIG['oggm_path'], path_list
    """
    results = pd.DataFrame(columns=['Original', 'temp05', 'prcpsum60', 'prcpwin60'])
    for region_dir in region_dirs:
        ds1 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist.nc')).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds2 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_2023tempminus05.nc')).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds3 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_2023prcpsumplus60.nc')).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds4 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_2023prcpwinplus60.nc')).sum(dim='rgi_id', skipna=True, keep_attrs=True)

        smb1 = calculate_smb_from_volume(ds1)
        smb2 = calculate_smb_from_volume(ds2)
        smb3 = calculate_smb_from_volume(ds3) #m w.s./yr
        smb4 = calculate_smb_from_volume(ds4)

        # ele1 = calculate_elevation_change(smb1)
        # ele2 = calculate_elevation_change(smb2)
        # ele3 = calculate_elevation_change(smb3)
        # ele4 = calculate_elevation_change(smb4)

        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[1]

        results.loc[region_name_clean, 'Original'] = smb1[-2]
        results.loc[region_name_clean, 'temp05'] = smb2[-2]
        results.loc[region_name_clean, 'prcpsum60'] = smb3[-2]
        results.loc[region_name_clean, 'prcpwin60'] = smb4[-2]

        ds1.close()
        ds2.close()
        ds3.close()
        ds4.close()

        print(f'{region_name} recorded.')

    results = results.sort_values(by='Original', ascending=True)
    fig, ax = plt.subplots(figsize=(8,12), dpi= 600)
    # x = np.arange(1,len(region_dirs))

    ax.scatter(results['Original'], results.index, marker='o', s=80, label='Original')
    ax.scatter(results['temp05'], results.index, marker='v', s=80, label='Temperature -0.5 °C')
    ax.scatter(results['prcpsum60'], results.index, marker='*', s=130, label='Summer precipitation +60 mm')
    ax.scatter(results['prcpwin60'], results.index, marker='x', s=80, label='Winter precipitation +60 mm')

    ax.tick_params(axis='both',labelsize=16)
    # ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Mass balance 2023 (m w.e./yr)', fontsize=18, labelpad=5)
    ax.grid(axis='y',linestyle='--', alpha=0.5,linewidth=0.3)
    ax.grid(axis='x',linestyle='--', alpha=1,linewidth=0.8)
    ax.legend(fontsize=15)

    save_path = os.path.join(save_dir, 'Sensitivity test', 'sensitivity test 2023.pdf')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=600, bbox_inches='tight')

    plt.show()
        
    return results


In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
save_dir = CONFIG['output_dir']

sensitivity_results = plot_sensitivity(region_dirs, save_dir, CONFIG)

## 7. 十年尺度气候极端分析（In[6]）

对比 2004-2013、2014-2023 两个十年段的冰川质量平衡统计，识别极端年份。


In [ ]:
def plot_decades_compar(region_dirs, save_dir, display_legend=False):
    """
    十年尺度对比：2005-2014 vs 2015-2024，计算均值与标准差并可视化。
    """
    for region_dir in region_dirs:
        ds1 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds2 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_warm_add_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds3 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_warm_rm_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        # mass balance
        smb1 = calculate_smb_from_volume(ds1) # normal
        smb2 = calculate_smb_from_volume(ds2) # 2005-2014 + warm
        smb3 = calculate_smb_from_volume(ds3) #m w.e./yr, 2015-2024 - warm

        # mean and std
        smb_pre = smb1[5:15]
        smb_cur = smb1[15:]
        smb_pre_warm = smb2[5:15] 
        smb_cur_dewarm = smb3[15:]

        mean_pre = np.mean(smb1[5:15])
        mean_cur = np.mean(smb1[15:])
        mean_pre_warm = np.mean(smb2[5:15])
        mean_cur_dewarm = np.mean(smb3[15:])

        std_pre = np.std(smb1[5:15])
        std_cur = np.std(smb1[15:])
        std_pre_warm = np.std(smb2[5:15])
        std_cur_dewarm = np.std(smb3[15:])

        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        
        # 迷你图
        fig, ax = plt.subplots(1,1,figsize=(4,3), dpi=600)
        t = ds1.time[:-1]

        ax.plot(t[15:], smb_cur, '^-', color=plt.cm.Accent(6), linewidth=1,markersize=3,label='Current decade')
        ax.plot(t[15:], smb_cur_dewarm, 'v-', color=plt.cm.Accent(7), linewidth=1,markersize=3,label='Current decade with warming trend removed')
        ax.plot(t[5:15], smb_pre, 's-', color=plt.cm.Accent(4), linewidth=1,markersize=3,label='Previous decade');
        ax.plot(t[5:15], smb_pre_warm, 'o-', color=plt.cm.Accent(5), linewidth=1,markersize=3,label='Previous decade with warming trend added');

        ax.hlines(mean_cur,2015,2024, linestyles='dashed',linewidth=2,color=plt.cm.Accent(6),zorder=5)
        ax.fill_between(t[15:], mean_cur-std_cur, mean_cur+std_cur,color=plt.cm.Accent(6),alpha = 0.11)
        ax.hlines(mean_cur_dewarm,2015,2024, linestyles='dashed',linewidth=2,color=plt.cm.Accent(7),zorder=5)
        ax.fill_between(t[15:],mean_cur_dewarm-std_cur_dewarm, mean_cur_dewarm+std_cur_dewarm,color=plt.cm.Accent(7),alpha = 0.11)
        ax.hlines(mean_pre,2005,2014, linestyles='dashed',linewidth=2,color=plt.cm.Accent(4),zorder=5)
        ax.fill_between(t[5:15],mean_pre-std_pre, mean_pre+std_pre,color=plt.cm.Accent(4),alpha = 0.11)
        ax.hlines(mean_pre_warm,2005,2014, linestyles='dashed',linewidth=2,color=plt.cm.Accent(5),zorder=5)
        ax.fill_between(t[5:15],mean_pre_warm-std_pre_warm, mean_pre_warm+std_pre_warm,color=plt.cm.Accent(5),alpha = 0.11)
        
        ax.set_title(region_name_clean.split(']')[1], fontsize=16, weight='bold')
        # ax.grid(axis='y',color='gray',linestyle=((0,(5,10))),linewidth = 0.3)
        ax.tick_params(labelsize=14)
        # ax.xaxis.set_major_locator(MultipleLocator(4))

        contribution = (1- ((mean_pre_warm - mean_pre) + (mean_cur - mean_cur_dewarm))/2/(mean_cur - mean_pre)) * 100
        ax.text(0.08,0.07,f'CR = {contribution:.1f}%',fontdict=dict(fontsize=16, color='k'),transform=ax.transAxes)

        if display_legend:
            ax.legend(fontsize=16, ncol=2, 
                bbox_to_anchor=(2, 1.2, 4, 2),
                loc='lower right',
                mode='expand')
    
        # plt.tight_layout()
        # 保存结果
        if display_legend:
            save_path = os.path.join(save_dir, 'pre_cur_decade_compar', 'legend', f'{region_name_clean}_legend.pdf')
        else:
            save_path = os.path.join(save_dir, 'pre_cur_decade_compar', f'{region_name_clean}.pdf')
        
        save_path = save_path.replace('/', '-')
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
        # plt.show()
        print(f'{region_name} finished! Most serious year: {np.argmin(smb1)+2000}')
    return None


In [ ]:
def plot_decades_compar(region_dirs, save_dir, display_legend=False):
    """
    十年尺度对比：2005-2014 vs 2015-2024，计算均值与标准差并可视化。
    """
    for region_dir in region_dirs:
        ds1 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds2 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_warm_add_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds3 = xr.open_dataset(os.path.join(region_dir, 'run_output_hist_warm_rm_spinup_all.nc'))\
                    .sel(time=slice(2000,2025)).sum(dim='rgi_id', skipna=True, keep_attrs=True)
        # mass balance
        smb1 = calculate_smb_from_volume(ds1) # normal
        smb2 = calculate_smb_from_volume(ds2) # 2005-2014 + warm
        smb3 = calculate_smb_from_volume(ds3) #m w.e./yr, 2015-2024 - warm

        # mean and std
        smb_pre = smb1[5:15]
        smb_cur = smb1[15:]
        smb_pre_warm = smb2[5:15] 
        smb_cur_dewarm = smb3[15:]

        mean_pre = np.mean(smb1[5:15])
        mean_cur = np.mean(smb1[15:])
        mean_pre_warm = np.mean(smb2[5:15])
        mean_cur_dewarm = np.mean(smb3[15:])

        std_pre = np.std(smb1[5:15])
        std_cur = np.std(smb1[15:])
        std_pre_warm = np.std(smb2[5:15])
        std_cur_dewarm = np.std(smb3[15:])

        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        
        # 迷你图
        fig, ax = plt.subplots(1,1,figsize=(7,4), dpi=600)
        t = ds1.time[:-1]

        ax.plot(t[15:], smb_cur, '^-', color=plt.cm.Accent(6), linewidth=1,markersize=3,label='Current decade')
        ax.plot(t[15:], smb_cur_dewarm, 'v-', color=plt.cm.Accent(7), linewidth=1,markersize=3,label='Current decade with warming trend removed')
        ax.plot(t[5:15], smb_pre, 's-', color=plt.cm.Accent(4), linewidth=1,markersize=3,label='Previous decade');
        ax.plot(t[5:15], smb_pre_warm, 'o-', color=plt.cm.Accent(5), linewidth=1,markersize=3,label='Previous decade with warming trend added');

        ax.hlines(mean_cur,2015,2024, linestyles='dashed',linewidth=2,color=plt.cm.Accent(6),zorder=5)
        ax.fill_between(t[15:], mean_cur-std_cur, mean_cur+std_cur,color=plt.cm.Accent(6),alpha = 0.11)
        ax.hlines(mean_cur_dewarm,2015,2024, linestyles='dashed',linewidth=2,color=plt.cm.Accent(7),zorder=5)
        ax.fill_between(t[15:],mean_cur_dewarm-std_cur_dewarm, mean_cur_dewarm+std_cur_dewarm,color=plt.cm.Accent(7),alpha = 0.11)
        ax.hlines(mean_pre,2005,2014, linestyles='dashed',linewidth=2,color=plt.cm.Accent(4),zorder=5)
        ax.fill_between(t[5:15],mean_pre-std_pre, mean_pre+std_pre,color=plt.cm.Accent(4),alpha = 0.11)
        ax.hlines(mean_pre_warm,2005,2014, linestyles='dashed',linewidth=2,color=plt.cm.Accent(5),zorder=5)
        ax.fill_between(t[5:15],mean_pre_warm-std_pre_warm, mean_pre_warm+std_pre_warm,color=plt.cm.Accent(5),alpha = 0.11)
        
        ax.set_title(region_name_clean.split(']')[1], fontsize=16, weight='bold')
        # ax.grid(axis='y',color='gray',linestyle=((0,(5,10))),linewidth = 0.3)
        ax.tick_params(labelsize=14)
        ax.xaxis.set_major_locator(MultipleLocator(4))

        contribution = (1- ((mean_pre_warm - mean_pre) + (mean_cur - mean_cur_dewarm))/2/(mean_cur - mean_pre)) * 100
        ax.text(0.08,0.07,f'CR = {contribution:.1f}%',fontdict=dict(fontsize=16, color='k'),transform=ax.transAxes)

        if display_legend:
            ax.legend(fontsize=16, ncol=2, 
                bbox_to_anchor=(2, 1.2, 4, 2),
                loc='lower right',
                mode='expand')
    
        # plt.tight_layout()
        # 保存结果
        if display_legend:
            save_path = os.path.join(save_dir, 'pre_cur_decade_compar', 'legend', f'{region_name_clean}_legend.pdf')
        else:
            save_path = os.path.join(save_dir, 'pre_cur_decade_compar', f'{region_name_clean}.pdf')
        
        save_path = save_path.replace('/', '-')
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
        # plt.show()
        print(f'{region_name} finished! Most serious year: {np.argmin(smb1)+2000}')
    return None


In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
save_dir = CONFIG['output_dir']

plot_decades_compar([region_dirs[3]], save_dir, display_legend=False)

In [ ]:
plot_decades_compar(region_dirs, save_dir, display_legend=False)

### 7.1 冰川消融与气候极端强度的非线性关系

In [ ]:
# 加载气候极端评分数据
score_path= r'E:\revised_NCC_data\OGGM model\climate_score\glacier_extreme_scores_ERA5.xlsx'

score_all = pd.read_excel(
    score_path, 
    sheet_name='Compound_Extreme_Scores', 
    header=0, 
    index_col=0
)
score_all.columns = score_all.columns.astype('int')

score_temp = pd.read_excel(
    score_path, 
    sheet_name='Temperature_Extreme_Scores', 
    header=0, 
    index_col=0
)
score_temp.columns = score_temp.columns.astype('int')

score_prcp = pd.read_excel(
    score_path, 
    sheet_name='Precipitation_Extreme_Scores', 
    header=0, 
    index_col=0
)
score_prcp.columns = score_prcp.columns.astype('int')

base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
region_names = [item.split(os.sep)[-1].split(']')[-1] for item in region_dirs]
region_names = [item.replace('-', '/') for item in region_names]

In [ ]:
def weight_average(array, weight):
    # 创建有效值掩码（排除 NaN 和对应的权重）
    valid_mask = ~np.isnan(array) & ~np.isnan(weight) & (weight > 0)
    result = np.average(
            array[valid_mask], 
            weights=weight[valid_mask])
    return result

def analyze_climate_extreme_impact(region_dirs):
    """
    分析气候极端事件对冰川质量平衡的影响
    
    参数:
        ds = xr.open_dataset(
            os.path.join(region_dir, 'run_output_hist.nc')
        )
        ds_total = ds.sum(dim='rgi_id', skipna=True, keep_attrs=True)
        
        pd.DataFrame: 包含各区域统计结果的数据框
    """
    
    # 2. 初始化结果数据框
    results = pd.DataFrame(
        data=np.zeros((12, len(region_names))), 
        columns=region_names
    )
    results.index = [
        'sum0410', 'sum1117', 'sum1824',  # 三个时间段的总质量平衡
        'avg0410', 'avg1117', 'avg1824',
        'ext0410', 'ext1117', 'ext1824',  # 极端年份的质量平衡
        'avg_ext0410', 'avg_ext1117', 'avg_ext1824'   # 极端年份数量
    ]
    
    # 3. 处理每个区域
    for region_dir in region_dirs:
        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[-1] # N/W Tien Shen

        # 加载OGGM数据
        ds = xr.open_dataset(
            os.path.join(region_dir, 'run_output_hist_spinup_all.nc')
        ).sel(time=slice(2000,2025))
        # ds_total = ds.sum(dim='rgi_id', skipna=True, keep_attrs=True)
        
        # # 计算总质量平衡
        # smb = calculate_smb_from_volume(ds_total) #2000-2024
        total_smb_sum_0410 = []
        total_smb_mean_0410 = []
        total_smb_sum_1117 = []
        total_smb_mean_1117 = []
        total_smb_sum_1824 = []
        total_smb_mean_1824 = []

        weight_0410 = []
        weight_1117 = []
        weight_1824 = []

        extreme_smb_sum_0410 = []
        extreme_smb_mean_0410 = []
        extreme_smb_sum_1117 = []
        extreme_smb_mean_1117 = []
        extreme_smb_sum_1824 = []
        extreme_smb_mean_1824 = []

        # 计算极端年总质量平衡和平均质量平衡
        for rgi_id in ds['rgi_id'].values:
            rgi_all_score = score_all.loc[rgi_id]
            rgi_smb = calculate_smb_from_volume(ds.sel(rgi_id=rgi_id)) #array，25个值, 2000-2024
            rgi_area = ds.sel(rgi_id=rgi_id)['area'].values

            weight_0410.append(np.min(rgi_area[4:11]))
            weight_1117.append(np.min(rgi_area[11:18]))
            weight_1824.append(np.min(rgi_area[18:25]))

            rgi_smb = np.where(np.abs(rgi_smb)>100, np.nan, rgi_smb)
            total_smb_sum_0410.append(np.nansum(rgi_smb[4:11]))
            total_smb_sum_1117.append(np.nansum(rgi_smb[11:18]))
            total_smb_sum_1824.append(np.nansum(rgi_smb[18:]))
            total_smb_mean_0410.append(np.nanmean(rgi_smb[4:11]))
            total_smb_mean_1117.append(np.nanmean(rgi_smb[11:18]))
            total_smb_mean_1824.append(np.nanmean(rgi_smb[18:]))
            # 极端年
            extreme_year = np.array(rgi_all_score.nlargest(5).index.tolist()) # (2012,2013,2022)
            
            extreme_year_filtered = extreme_year[(extreme_year >= 2004) & (extreme_year <= 2010)]
            if len(extreme_year_filtered) > 0:
                temp_smb = rgi_smb[extreme_year_filtered-2000]
                temp_sum_0410 = np.nansum(temp_smb)
                temp_mean_0410 = np.nanmean(temp_smb)
            else:
                temp_sum_0410 = np.nan
                temp_mean_0410 = np.nan
            
            extreme_year_filtered = extreme_year[(extreme_year >= 2011) & (extreme_year <= 2017)]
            if len(extreme_year_filtered) > 0:
                temp_smb = rgi_smb[extreme_year_filtered-2000]
                temp_sum_1117 = np.nansum(temp_smb)
                temp_mean_1117 = np.nanmean(temp_smb)
            else:
                temp_sum_1117 = np.nan
                temp_mean_1117 = np.nan
            
            extreme_year_filtered = extreme_year[(extreme_year >= 2018) & (extreme_year <= 2024)]
            if len(extreme_year_filtered) > 0:
                temp_smb = rgi_smb[extreme_year_filtered-2000]
                temp_sum_1824 = np.nansum(temp_smb)
                temp_mean_1824 = np.nanmean(temp_smb)
            else:
                temp_sum_1824 = np.nan
                temp_mean_1824 = np.nan

            extreme_smb_sum_0410.append(temp_sum_0410)
            extreme_smb_mean_0410.append(temp_mean_0410)
            extreme_smb_sum_1117.append(temp_sum_1117)
            extreme_smb_mean_1117.append(temp_mean_1117)
            extreme_smb_sum_1824.append(temp_sum_1824)
            extreme_smb_mean_1824.append(temp_mean_1824)

        total_smb_mean_0410 = np.where(np.abs(total_smb_sum_0410)>500, np.nan, total_smb_mean_0410) # all rgis
        total_smb_sum_0410 = np.where(np.abs(total_smb_sum_0410)>500, np.nan, total_smb_sum_0410)
        total_smb_mean_1117 = np.where(np.abs(total_smb_sum_1117)>500, np.nan, total_smb_mean_1117)
        total_smb_sum_1117 = np.where(np.abs(total_smb_sum_1117)>500, np.nan, total_smb_sum_1117)
        total_smb_mean_1824 = np.where(np.abs(total_smb_sum_1824)>500, np.nan, total_smb_mean_1824)
        total_smb_sum_1824 = np.where(np.abs(total_smb_sum_1824)>500, np.nan, total_smb_sum_1824)

        extreme_smb_mean_0410 = np.where(np.abs(extreme_smb_sum_0410)>500, np.nan, extreme_smb_mean_0410)
        extreme_smb_sum_0410 = np.where(np.abs(extreme_smb_sum_0410)>500, np.nan, extreme_smb_sum_0410)
        extreme_smb_mean_1117 = np.where(np.abs(extreme_smb_sum_1117)>500, np.nan, extreme_smb_mean_1117)
        extreme_smb_sum_1117 = np.where(np.abs(extreme_smb_sum_1117)>500, np.nan, extreme_smb_sum_1117)
        extreme_smb_mean_1824 = np.where(np.abs(extreme_smb_sum_1824)>500, np.nan, extreme_smb_mean_1824)
        extreme_smb_sum_1824 = np.where(np.abs(extreme_smb_sum_1824)>500, np.nan, extreme_smb_sum_1824)

        weight_0410 = np.array(weight_0410)
        weight_1117 = np.array(weight_1117)
        weight_1824 = np.array(weight_1824)
        
        # 计算统计量
        results.loc['sum0410', region_name_clean] = weight_average(total_smb_sum_0410, weight_0410)
        results.loc['sum1117', region_name_clean] = weight_average(total_smb_sum_1117, weight_1117)
        results.loc['sum1824', region_name_clean] = weight_average(total_smb_sum_1824, weight_1824)
        results.loc['avg0410', region_name_clean] = weight_average(total_smb_mean_0410, weight_0410)
        results.loc['avg1117', region_name_clean] = weight_average(total_smb_mean_1117, weight_1117)
        results.loc['avg1824', region_name_clean] = weight_average(total_smb_mean_1824, weight_1824)

        results.loc['ext0410', region_name_clean] = weight_average(extreme_smb_sum_0410, weight_0410)
        results.loc['ext1117', region_name_clean] = weight_average(extreme_smb_sum_1117, weight_1117)
        results.loc['ext1824', region_name_clean] = weight_average(extreme_smb_sum_1824, weight_1824)
        results.loc['avg_ext0410', region_name_clean] = weight_average(extreme_smb_mean_0410, weight_0410)
        results.loc['avg_ext1117', region_name_clean] = weight_average(extreme_smb_mean_1117, weight_1117)
        results.loc['avg_ext1824', region_name_clean] = weight_average(extreme_smb_mean_1824, weight_1824)
        
        print(f"✓ 处理完成: {region_name_clean}")

    return results


In [ ]:
results = analyze_climate_extreme_impact(region_dirs)
results

In [ ]:
results.to_excel(r'E:\revised_NCC_data\OGGM model\climate_score\three_periods_smb_statistic.xlsx')

### 7.2 三个7年对比（04-10，11-17，18-24）

In [ ]:
results = results.T  # row_index: region_name
results = results.sort_values(by='sum1824', ascending=True)

results['per0410'] = results['ext0410']/results['sum0410']
results['per1117'] = results['ext1117']/results['sum1117']
results['per1824'] = results['ext1824']/results['sum1824']

In [ ]:
def plot_climate_extreme_comparison(results, save_path=None, display_legend=False):
    """
    绘制气候极端事件对冰川质量平衡影响的对比图
    
    参数:
        results_path: str, 结果数据Excel文件路径
        save_path: str, 保存路径（可选）
    """

    region_names = results.index
    y_pos = np.arange(len(region_names))
    
    # 2. 创建图形
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 17), dpi=600)
    
    # 3. 绘制子图1：累积质量平衡对比
    # 绘制总质量平衡（背景条）
    ax1.barh(y_pos - 0.3, results['sum1824'], height=0.3, align='center',
             color=plt.cm.Accent(3), edgecolor='k', label='2018-2024')
    ax1.barh(y_pos, results['sum1117'], height=0.3, align='center',
             color=plt.cm.Accent(2), edgecolor='k', label='2011-2017')
    ax1.barh(y_pos + 0.3, results['sum0410'], height=0.3, align='center',
             color=plt.cm.Accent(1), edgecolor='k', label='2004-2010')
    
    # 绘制极端年份质量平衡（前景条）
    ax1.barh(y_pos - 0.3, results['ext1824'], height=0.3, align='center',
             color=plt.cm.Accent(6), label='2018-2024 extreme years')
    ax1.barh(y_pos, results['ext1117'], height=0.3, align='center',
             color=plt.cm.Accent(5), label='2011-2017 extreme years')
    ax1.barh(y_pos + 0.3, results['ext0410'], height=0.3, align='center',
             color=plt.cm.Accent(4), label='2004-2010 extreme years')
    
    # 添加百分比标签（2018-2024）
    for i, region_name in enumerate(region_names):
        if (results.loc[region_name, 'per1824'] > 0) and (results.loc[region_name, 'per1824'] < 1) and (results.loc[region_name, 'ext1824'] > 0):
            ax1.text(results.loc[region_name, 'ext1824'] + 1.2, y_pos[i] - 0.43,
                    f'{results.loc[region_name, 'per1824']:.0%}',
                    color=plt.cm.Accent(6), fontsize=14, fontweight='bold')
        elif (results.loc[region_name, 'per1824'] > 0) and (results.loc[region_name, 'per1824'] < 1) and (results.loc[region_name, 'ext1824'] < 0):
            ax1.text(results.loc[region_name, 'ext1824'] - 0.2, y_pos[i] - 0.43,
                    f'{results.loc[region_name, 'per1824']:.0%}',
                    color=plt.cm.Accent(6), fontsize=14, fontweight='bold')
    
    # 添加百分比标签（2011-2017）
    for i, region_name in enumerate(region_names):
        if (results.loc[region_name, 'per1117'] > 0) and (results.loc[region_name, 'per1117'] < 1) and (results.loc[region_name, 'ext1117'] > 0):
            ax1.text(results.loc[region_name, 'ext1117'] + 1.2, y_pos[i] - 0.13,
                    f'{results.loc[region_name, 'per1117']:.0%}',
                    color=plt.cm.Accent(5), fontsize=14, fontweight='bold')
        elif (results.loc[region_name, 'per1117'] > 0) and (results.loc[region_name, 'per1117'] < 1) and (results.loc[region_name, 'ext1117'] < 0):
            ax1.text(results.loc[region_name, 'ext1117'] - 0.15, y_pos[i] - 0.13,
                    f'{results.loc[region_name, 'per1117']:.0%}',
                    color=plt.cm.Accent(5), fontsize=14, fontweight='bold')
    
    # 添加百分比标签（2004-2010）
    for i, region_name in enumerate(region_names):
        if (results.loc[region_name, 'per0410'] > 0) and (results.loc[region_name, 'per0410'] < 1) and (results.loc[region_name, 'ext0410'] > 0):
            ax1.text(results.loc[region_name, 'ext0410'] + 1.2, y_pos[i] + 0.17,
                    f'{results.loc[region_name, 'per0410']:.0%}',
                    color=plt.cm.Accent(4), fontsize=14, fontweight='bold')
        elif (results.loc[region_name, 'per0410'] > 0) and (results.loc[region_name, 'per0410'] < 1) and (results.loc[region_name, 'ext0410'] < 0):
            ax1.text(results.loc[region_name, 'ext0410'] - 0.1, y_pos[i] + 0.17,
                    f'{results.loc[region_name, 'per0410']:.0%}',
                    color=plt.cm.Accent(4), fontsize=14, fontweight='bold')
    
    # 设置子图1的坐标轴和标签
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(region_names, fontdict=dict(fontsize=16, color='black', 
                                              family='Arial', weight='normal', alpha=1.0))
    ax1.set_xlabel('m w.e.', fontsize=18, labelpad=5)
    ax1.tick_params(axis='x', labelsize=16)
    ax1.set_title('a Cumulative mass balance',
                 fontdict=dict(fontsize=18, color='black', family='Arial', 
                              weight='bold', alpha=1.0), pad=10)

    # ax1.grid(linestyle='--', alpha=0.5, linewidth=0.8)
    
    # 添加分隔线
    lim1 = ax1.get_xlim()
    for i in range(len(region_names)+1):
        ax1.hlines(i - 0.5, lim1[0], lim1[1], linestyles='dashed',
                  linewidth=1.5, color='k', alpha=0.6, zorder=5)
    # ax1.set_xlim(lim1[0], lim1[1])
    ax1.set_ylim(-1, len(region_names))
    ax1.invert_xaxis()
    
    # 4. 绘制子图2：极端年份平均质量平衡
    hbars1 = ax2.barh(y_pos - 0.3, results['avg_ext1824'], height=0.3, align='center',
                     color=plt.cm.Accent(6), label='2018-2024 extreme years')
    hbars2 = ax2.barh(y_pos, results['avg_ext1117'], height=0.3, align='center',
                     color=plt.cm.Accent(5), label='2011-2017 extreme years')
    hbars3 = ax2.barh(y_pos + 0.3, results['avg_ext0410'], height=0.3, align='center',
                     color=plt.cm.Accent(4), label='2004-2010 extreme years')
    # 总体平均质量平衡
    ax2.barh(y_pos - 0.3, results['avg1824'], height=0.3, align='center',
                     color=plt.cm.Accent(3), edgecolor='k', label='2018-2024')
    ax2.barh(y_pos, results['avg1117'], height=0.3, align='center',
                     color=plt.cm.Accent(2), edgecolor='k', label='2011-2017')
    ax2.barh(y_pos + 0.3, results['avg0410'], height=0.3, align='center',
                     color=plt.cm.Accent(1), edgecolor='k', label='2004-2010')
    
    # # 添加标签（极端年份数量）
    # ax2.bar_label(hbars1, labels=[f'{r:.0f}' for r in results['len1824']],
    #              padding=-8, color=plt.cm.Accent(6), fontsize=14)
    # ax2.bar_label(hbars2, labels=[f'{r:.0f}' for r in results['len1117']],
    #              padding=-8, color=plt.cm.Accent(5), fontsize=14)
    # ax2.bar_label(hbars3, labels=[f'{r:.0f}' for r in results['len0010']],
    #              padding=-8, color=plt.cm.Accent(4), fontsize=14)
    
    # 设置子图2的坐标轴
    ax2.yaxis.set_major_locator(MultipleLocator(1))
    ax2.yaxis.set_visible(False)
    ax2.set_xlabel('m w.e./yr', fontsize=18, labelpad=5)
    ax2.tick_params(axis='x', labelsize=16)
    ax2.set_title('b Mean mass balance',
                 fontdict=dict(fontsize=18, color='black', family='Arial', 
                              weight='bold', alpha=1.0), pad=10)
    # ax2.legend()
    # ax2.grid(linestyle='--', alpha=0.5, linewidth=0.8)
    
    # 添加分隔线
    lim2 = ax2.get_xlim()
    for i in range(len(region_names)):
        ax2.hlines(i - 0.5, lim2[0], lim2[1], linestyles='dashed',
                  linewidth=1.5, color='k', alpha=0.6, zorder=5)
    ax2.set_xlim(lim2[0], lim2[1])
    ax2.set_ylim(-1, len(region_names))
    ax2.invert_xaxis()

    if display_legend:
        ax1.legend(fontsize=16, ncol=3, 
            bbox_to_anchor=(0, -0.12, 2, 0.1),
            loc='lower right',
            mode='expand')
    
    # 5. 调整布局
    plt.subplots_adjust(wspace=0.1)
    
    # 6. 保存或显示
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()
    return None

In [ ]:
save_path = os.path.join(CONFIG['output_dir'], 'three_period_compar_bar', 'three_period_compar_bar.pdf')
os.makedirs(os.path.dirname(save_path), exist_ok=True)

plot_climate_extreme_comparison(results, save_path=save_path, display_legend=True)

### 7.3 前后十年对比散点图

In [ ]:
def analyze_climate_extreme_impact2(region_dirs):
    # 2. 初始化结果数据框
    results = pd.DataFrame(
        data=np.zeros((8, len(region_names))), 
        columns=region_names
    )

    results.index = [
        'sum0514', 'sum1524',  # 三个时间段的总质量平衡
        'avg0514', 'avg1524', 
        'ext0514', 'ext1524',  # 极端年份的质量平衡
        'avg_ext0514', 'avg_ext1524'   # 极端年份数量
    ]
    
    # 3. 处理每个区域
    for region_dir in region_dirs:
        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[-1] # N/W Tien Shen

        # 加载OGGM数据
        ds = xr.open_dataset(
            os.path.join(region_dir, 'run_output_hist_spinup_all.nc')
        ).sel(time=slice(2000,2025))
        
        # 计算质量平衡
        # smb = calculate_smb_from_volume(ds) #2000-2024
        
        # 合并数据
        total_smb_sum_0514 = []
        total_smb_mean_0514 = []
        total_smb_sum_1524 = []
        total_smb_mean_1524 = []

        weight_0514 = []
        weight_1524 = []

        extreme_smb_sum_0514 = []
        extreme_smb_mean_0514 = []
        extreme_smb_sum_1524 = []
        extreme_smb_mean_1524 = []
        
        # 计算极端年总质量平衡和平均质量平衡
        for rgi_id in ds['rgi_id'].values:
            rgi_all_score = score_all.loc[rgi_id]
            rgi_smb = calculate_smb_from_volume(ds.sel(rgi_id=rgi_id)) #array，25个值
            rgi_area = ds.sel(rgi_id=rgi_id)['area'].values

            weight_0514.append(np.min(rgi_area[5:15]))
            weight_1524.append(np.min(rgi_area[15:25]))

            rgi_smb = np.where(np.abs(rgi_smb)>100, np.nan, rgi_smb)
            total_smb_sum_0514.append(np.nansum(rgi_smb[5:15]))
            total_smb_sum_1524.append(np.nansum(rgi_smb[15:]))
            total_smb_mean_0514.append(np.nanmean(rgi_smb[5:15]))
            total_smb_mean_1524.append(np.nanmean(rgi_smb[15:]))
            # 极端年
            extreme_year = np.array(rgi_all_score.nlargest(5).index.tolist()) # (2012,2013,2022)
            
            extreme_year_filtered = extreme_year[(extreme_year >= 2005) & (extreme_year <= 2014)]
            if len(extreme_year_filtered) > 0:
                temp_smb = rgi_smb[extreme_year_filtered-2000]
                temp_sum_0514 = np.nansum(temp_smb)
                temp_mean_0514 = np.nanmean(temp_smb)
            else:
                temp_sum_0514 = np.nan
                temp_mean_0514 = np.nan
            
            extreme_year_filtered = extreme_year[(extreme_year >= 2015) & (extreme_year <= 2024)]
            if len(extreme_year_filtered) > 0:
                temp_smb = rgi_smb[extreme_year_filtered-2000]
                temp_sum_1524 = np.nansum(temp_smb)
                temp_mean_1524 = np.nanmean(temp_smb)
            else:
                temp_sum_1524 = np.nan
                temp_mean_1524 = np.nan

            extreme_smb_sum_0514.append(temp_sum_0514)
            extreme_smb_mean_0514.append(temp_mean_0514)
            extreme_smb_sum_1524.append(temp_sum_1524)
            extreme_smb_mean_1524.append(temp_mean_1524)

        total_smb_mean_0514 = np.where(np.abs(total_smb_sum_0514)>500, np.nan, total_smb_mean_0514)
        total_smb_sum_0514 = np.where(np.abs(total_smb_sum_0514)>500, np.nan, total_smb_sum_0514)
        total_smb_mean_1524 = np.where(np.abs(total_smb_sum_1524)>500, np.nan, total_smb_mean_1524)
        total_smb_sum_1524 = np.where(np.abs(total_smb_sum_1524)>500, np.nan, total_smb_sum_1524)

        extreme_smb_mean_0514 = np.where(np.abs(extreme_smb_sum_0514)>500, np.nan, extreme_smb_mean_0514)
        extreme_smb_sum_0514 = np.where(np.abs(extreme_smb_sum_0514)>500, np.nan, extreme_smb_sum_0514)
        extreme_smb_mean_1524 = np.where(np.abs(extreme_smb_sum_1524)>500, np.nan, extreme_smb_mean_1524)
        extreme_smb_sum_1524 = np.where(np.abs(extreme_smb_sum_1524)>500, np.nan, extreme_smb_sum_1524)

        weight_0514 = np.array(weight_0514)
        weight_1524 = np.array(weight_1524)
        
        # 计算统计量
        results.loc['sum0514', region_name_clean] = weight_average(total_smb_sum_0514, weight_0514)
        results.loc['sum1524', region_name_clean] = weight_average(total_smb_sum_1524, weight_1524)
        results.loc['avg0514', region_name_clean] = weight_average(total_smb_mean_0514, weight_0514)
        results.loc['avg1524', region_name_clean] = weight_average(total_smb_mean_1524, weight_1524)

        results.loc['ext0514', region_name_clean] = weight_average(extreme_smb_sum_0514, weight_0514)
        results.loc['ext1524', region_name_clean] = weight_average(extreme_smb_sum_1524, weight_1524)
        results.loc['avg_ext0514', region_name_clean] = weight_average(extreme_smb_mean_0514, weight_0514)
        results.loc['avg_ext1524', region_name_clean] = weight_average(extreme_smb_mean_1524, weight_1524)
        
        print(f"✓ 处理完成: {region_name_clean}")

    return results

results = analyze_climate_extreme_impact2(region_dirs)

results = results.T  # row_index: region_name
results = results.sort_values(by='sum1524', ascending=True)

results['per0514'] = results['ext0514']/results['sum0514']
results['per1524'] = results['ext1524']/results['sum1524']

results.to_excel(r'E:\revised_NCC_data\OGGM model\climate_score\two_period_smb_statistic.xlsx')
results

In [ ]:
def plot_10yr_climate_extreme_scatter(results, save_path=None):
    """
    绘制十年尺度气候极端事件对冰川质量平衡影响的散点图
    
    参数:
        results_path: str, 结果数据Excel文件路径
        save_path: str, 保存路径（可选）
    """
    region_names = results.index
    y_pos = np.arange(len(region_names))

    # 2. 根据数值正负创建颜色列（向量化操作）
    results['color_sum0514'] = ['red' if y < 0 else 'blue' for y in results['sum0514']]
    results['color_sum1524'] = ['red' if y < 0 else 'blue' for y in results['sum1524']]
    results['color_ext0514'] = ['red' if y < 0 else 'blue' for y in results['ext0514']]
    results['color_ext1524'] = ['red' if y < 0 else 'blue' for y in results['ext1524']]
    
    label = ['Total', 'Extreme years']
    
    # 3. 创建图形
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(17, 22), dpi=600)
    
    # 4. 子图1：2004-2013累积质量平衡
    ax1.scatter(np.zeros(len(region_names)), y_pos, 
               s=500*(np.abs(results['sum0514'])**0.5), 
               c=results['color_sum0514'], alpha=0.5)
    ax1.scatter(np.zeros(len(region_names))+1, y_pos, 
               s=500*(np.abs(results['ext0514'])**0.5), 
               c=results['color_ext0514'], alpha=1)
    
    ax1.set_xticks([0, 1])
    ax1.set_xticklabels(label, fontdict=dict(fontsize=18, color='black', 
                                             family='Arial', weight='bold', alpha=1.0))
    ax1.tick_params(axis='x', which='major', pad=13, width=3, length=7)

    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(region_names, fontdict=dict(fontsize=18, color='black', 
                                              family='Arial', weight='bold', alpha=1.0))
    ax1.tick_params(axis='y', width=3, length=8, pad=5)

    ax1.set_title('a Cumulative mass balance \n (2005 - 2014, m w.e.)',
                 fontdict=dict(fontsize=20, color='black', family='Arial', 
                              weight='bold', alpha=1.0), pad=15)
    ax1.set_xlim(-0.5, 2.1)
    ax1.set_ylim(-0.8, len(region_names)-0.2)
    # ax1.grid(linestyle='--', alpha=0.5, linewidth=0.3)
    ax1.invert_yaxis()
    
    # 添加数值标签和百分比
    for i, region_name in enumerate(region_names):
        ax1.text(0.2, y_pos[i]+0.15, f'{results.loc[region_name, 'sum0514']:.1f}',
                color='k', fontsize=17, fontweight='bold')
        ax1.text(1.2, y_pos[i]+0.15, f'{results.loc[region_name, 'ext0514']:.1f}',
                color='k', fontsize=17, fontweight='bold')
        if (results.loc[region_name, 'per0514'] > 0) and (results.loc[region_name, 'per0514'] < 1.5):
            ax1.text(1.6, y_pos[i]+0.15, f'{results.loc[region_name, 'per0514']:.1%}',
                    color='red', fontsize=17, fontweight='bold')
    
    # 5. 子图2：2014-2023累积质量平衡
    ax2.scatter(np.zeros(len(region_names)), y_pos, 
               s=500*(np.abs(results['sum1524'])**0.5), 
               c=results['color_sum1524'], alpha=0.6)
    ax2.scatter(np.zeros(len(region_names))+1, y_pos, 
               s=500*(np.abs(results['ext1524'])**0.5), 
               c=results['color_ext1524'], alpha=1)
    
    ax2.set_xticks([0, 1])
    ax2.set_xticklabels(label, fontdict=dict(fontsize=18, color='black', 
                                             family='Arial', weight='bold', alpha=1.0))
    ax2.tick_params(axis='x', which='major', pad=13, width=3, length=7)

    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([])
    ax2.tick_params(axis='y', width=3, length=8)
    ax2.set_title('b Cumulative mass balance \n (2015 - 2024, m w.e.)',
                 fontdict=dict(fontsize=20, color='black', family='Arial', 
                              weight='bold', alpha=1.0), pad=15)
    ax2.set_xlim(-0.5, 2.1)
    ax2.set_ylim(-0.8, len(region_names)-0.2)
    # ax2.grid(linestyle='--', alpha=0.5, linewidth=0.3)
    ax2.invert_yaxis()
    
    # 添加数值标签和百分比
    for i, region_name in enumerate(region_names):
        ax2.text(0.2, y_pos[i]+0.15, f'{results.loc[region_name, 'sum1524']:.1f}',
                color='k', fontsize=17, fontweight='bold')
        ax2.text(1.2, y_pos[i]+0.15, f'{results.loc[region_name, 'ext1524']:.1f}',
                color='k', fontsize=17, fontweight='bold')
        if (results.loc[region_name, 'per1524'] > 0) and (results.loc[region_name, 'per1524'] < 1.5):
            ax2.text(1.6, y_pos[i]+0.15, f'{results.loc[region_name, 'per1524']:.1%}',
                    color='red', fontsize=17, fontweight='bold')
    
    # 6. 子图3：极端年份平均质量平衡
    hbars1 = ax3.barh(y_pos-0.175, results['avg_ext0514'], height=0.3, align='center',
                     color=plt.cm.Accent(1), label='2005-2014')
    hbars2 = ax3.barh(y_pos+0.175, results['avg_ext1524'], height=0.3, align='center',
                     color=plt.cm.Accent(5), label='2015-2024')
    
    ax3.invert_xaxis()
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels([])
    ax3.tick_params(axis='y', width=3, length=8)
    ax3.tick_params(axis='x', width=3, length=7)
    ax3.tick_params(labelsize=18)

    # ax3.set_xlabel('Mean mass balance (m w.e./yr)', fontsize=16, 
    #                fontweight='bold', labelpad=5)
    ax3.set_title('c Mean mass balance of\n extreme years (m w.e./yr)',
                 fontdict=dict(fontsize=20, color='black', family='Arial', 
                              weight='bold', alpha=1.0), pad=15)
    # ax3.grid(linestyle='--', alpha=0.5, linewidth=0.3)
    ax3.set_ylim(-0.8, len(region_names)-0.2)
    ax3.invert_yaxis()
    
    # 添加标签（极端年份数量）
#     ax3.bar_label(hbars1, labels=[f'{r:.0f}' for r in results['len0514']], 
#                  padding=-10, color='black', fontsize=16, fontweight='bold')
#     ax3.bar_label(hbars2, labels=[f'{r:.0f}' for r in results['len1524']], 
#                  padding=-10, color='black', fontsize=16, fontweight='bold')
    
    # 7. 保存或显示
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    
    plt.show()


In [ ]:
save_path = os.path.join(CONFIG['output_dir'], 'two_period_compar_scatter', 'two_period_compar_scatter.pdf')
os.makedirs(os.path.dirname(save_path), exist_ok=True)

plot_10yr_climate_extreme_scatter(results, save_path=save_path)

## 8. 未来气候投影对比（In[7]）

对比OGGM不同投影场景与CMIP6（示例使用EC-Earth3 SSP245）的体积/面积/高程变化，评估一致性与不确定性。


In [ ]:
def plot_future_projection_comparison(region_dirs, plot_type='elevation', save_dir=None, display_legend=False):
    """
    绘制未来气候投影对比图：历史模拟与多种未来情景的对比
    
    参数:
        oggm_path: str, OGGM数据基础路径
        plot_type: str, 绘制类型，可选 'elevation'（高程）、'volume'（体积）、'area'（面积）
        save_dir: str, 保存目录（可选）
    """
    # 定义文件列表
    file_suffixes = [
        'run_output_hist_spinup_all.nc',                    # ds1: 历史模拟
        'run_output_cmip6_normal_spinup_all.nc', 
        'run_output_cmip6_repu_his_extremes_QDM_spinup_all.nc',
    ]
    cmip6_scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

    # 绘图配置
    plot_configs = [
        {'color': 'b', 'label': 'Historical simulation', 'time_slice': slice(0, 25)},
        {'color': plt.cm.Accent(0), 'label': 'SSP126', 'time_slice': slice(25, 101)},
        {'color': plt.cm.Accent(2), 'label': 'SSP245', 'time_slice': slice(25, 101)},
        {'color': plt.cm.Accent(6), 'label': 'SSP370', 'time_slice': slice(25, 101)},
        {'color': 'red', 'label': 'SSP585', 'time_slice': slice(25, 101)}
    ]
    
    # 根据绘制类型设置参数
    plot_params = {
        'elevation': {
            'unit_scale': 1.0,
            'ylabel': 'Glacier elevation Change (m)',
            'text_prefix': 'Elevation Change',
            'calc_method': 'difference',  # elechange1 = ele2[0] - ele2[23]
            'subfolder': 'pr_ele'
        },
        'volume': {
            'unit_scale': 1e-9,  # 转换为 km³
            'ylabel': 'Glacier volume (km³)',
            'text_prefix': 'Volume loss',
            'calc_method': 'percentage',  # ablation = 1 - vol2[23]/vol2[0]
            'subfolder': 'pr_vol'
        },
        'area': {
            'unit_scale': 1e-6,  # 转换为 km²
            'ylabel': 'Glacier area (km²)',
            'text_prefix': 'Area loss',
            'calc_method': 'percentage',  # ablation = 1 - are2[23]/are2[0]
            'subfolder': 'pr_area'
        }
    }
    
    params = plot_params[plot_type]
    
    for region_dir in region_dirs:
        # 1. 加载所有数据集
        datasets = []
        std_datasets = [] # uncertainty, * sqrt(n)
        std2_datasets = [] 
        plot_attrs = []
        
        base_file = os.path.join(region_dir, file_suffixes[0])
        # hist file
        ds_hist = xr.open_dataset(base_file).sel(time=slice(2000, 2025))
        n = len(ds_hist.rgi_id)
        ds_sum = ds_hist.sum(dim='rgi_id', skipna=True, keep_attrs=True)
        ds_err = ds_hist.std(dim='rgi_id', skipna=True, keep_attrs=True) * np.sqrt(n)
        ds_err2 = ds_hist.std(dim='rgi_id', skipna=True, keep_attrs=True)

        datasets.append(ds_sum)
        std_datasets.append(ds_err)
        plot_attrs.append(plot_configs[0])
        std2_datasets.append(ds_err2)
        
        # Load projection files
        for suffix in file_suffixes[1:]:
            file_path = os.path.join(region_dir, suffix)
            ds_proj = xr.open_dataset(file_path).sel(time=slice(2000, 2101))
            n = len(ds_proj.rgi_id)
            
            for ssp_i, ssp in enumerate(cmip6_scenarios):
                ds_sum = ds_proj.sel(SSP=ssp).sum(dim='rgi_id', skipna=True, keep_attrs=True) #所有冰川的面积/体积
                ds_err = ds_proj.sel(SSP=ssp).std(dim='rgi_id', skipna=True, keep_attrs=True) * np.sqrt(n) #所有冰川的不确定性，总体
                ds_err2 = ds_proj.sel(SSP=ssp).std(dim='rgi_id', skipna=True, keep_attrs=True)

                datasets.append(ds_sum)
                std_datasets.append(ds_err)
                plot_attrs.append(plot_configs[ssp_i+1])
                std2_datasets.append(ds_err2)
        
        # 2. 根据绘制类型计算数据
        data_list = []
        error_list = []
        
        if plot_type == 'elevation':
            # 计算质量平衡和高程变化
            for ds, std in zip(datasets, std_datasets):            
                # 计算质量平衡
                smb = calculate_smb_from_volume(ds) #2000全年-2100全年，通量
                ele = calculate_elevation_change(smb) #2001/1-2101/1

                smb_err = (std['volume'].values[1:])**2 + (std['volume'].values[:-1])**2 + (smb * std['area'].values[1:])**2
                smb_err = np.sqrt(smb_err) / ds['area'].values[1:] # m
                smb_err = np.where((smb_err>1e2) | (np.isnan(smb_err)), np.nanmedian(smb_err), smb_err)

                ele_err = smb_err/0.9 # ice elevation, m
                # ele_err = np.sqrt(np.cumsum(ele_err**2))

                data_list.append(ele)
                error_list.append(ele_err)
        
        else:  # volume 或 area
            var_name = 'volume' if plot_type == 'volume' else 'area'
            for ds, std in zip(datasets, std_datasets):
                # 提取数据并应用单位转换
                data = ds[var_name][:-1].values * params['unit_scale'] #2000/1/1-2100/1/1，状态量
                error = std[var_name][:-1].values * params['unit_scale']
                
                data_list.append(data)
                error_list.append(error)
        
        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[1]

        # 3. 绘图
        title = region_name_clean
        fig, ax = plt.subplots(1, 1, figsize=(5, 3), dpi=600)
        
        # 绘制所有情景
        for i, (ds, data, error, config) in enumerate(zip(datasets, data_list, error_list, plot_attrs)):
            time_slice = config['time_slice']

            time_data = ds.time[time_slice]
            data_plot = data[time_slice]
            error_plot = error[time_slice]
            
            if (i==0) or (i>=5): # QDM
                ax.fill_between(time_data, data_plot - error_plot, data_plot + error_plot,
                                color=config['color'], alpha=0.15)
                ax.plot(time_data, data_plot, color=config['color'], 
                        linewidth=3 if i == 0 else 2, label=config['label'], zorder=5)
            else: # CMIP6 Normal
                ax.fill_between(time_data, data_plot - error_plot, data_plot + error_plot,
                                color=config['color'], alpha=0.15)
                ax.plot(time_data, data_plot, color=config['color'], linestyle='--',
                        linewidth=3 if i == 0 else 2, zorder=5)
        
        # 添加垂直线标记
        bound = ax.get_ylim()
        ax.vlines(2024, bound[0] + 0.7*(bound[1] - bound[0]), bound[1],
                    linestyles='dashed', linewidth=1.5, color='k', zorder=5)
        # ax.vlines(2100, bound[0], bound[1],
        #             linestyles='dashed', linewidth=1.5, color='k', zorder=5)
        ax.set_ylim(bound)
        
        # 添加50%线（仅对volume和area）
        if plot_type in ['volume', 'area']:
            half_value = data_list[0][0] / 2
            ax.hlines(half_value, 2000, 2100, linestyles='dashed', 
                        linewidth=1.5, color=plt.cm.Accent(5), zorder=5)
        
        ax.tick_params(labelsize=14)
        data2 = data_list[-1] #最严重的， SSP585，QDM
        
        if params['calc_method'] == 'difference':
            # 高程变化：直接差值
            change1 = -data2[24]
            change2 = data2[24] - data2[99]
            text1 = f"{params['text_prefix']}$_{{2000-2024}}$ = {change1:.1f} m"
            text2 = f"{params['text_prefix']}$_{{2025-2100}}$ = {change2:.1f} m"
        else:
            # 体积/面积损失：百分比
            ablation1 = 1 - data2[25] / data2[0]
            ablation2 = (data2[25] - data2[100]) / data2[0]
            text1 = f"{params['text_prefix']}$_{{2000-2024}}$ = {ablation1:.1%}"
            text2 = f"{params['text_prefix']}$_{{2025-2100}}$ = {ablation2:.1%}"
        
        ax.text(0.05, 0.16, text1,
                fontdict=dict(fontsize=15, color='k'), transform=ax.transAxes, zorder=6)
        ax.text(0.05, 0.08, text2,
                fontdict=dict(fontsize=15, color='k'), transform=ax.transAxes, zorder=6)
        
        # 设置标题和格式
        ax.set_title(title, pad=5, fontdict=dict(fontsize=16, color='black', 
                                                family='Arial', weight='bold', alpha=1.0))
        # ax.grid(axis='both', color='gray', linestyle=((0, (5, 10))), linewidth=0.3)
        # ax.xaxis.set_major_locator(MultipleLocator(base=20))

        if plot_type in ['volume', 'area']:
            for i in range(1, len(datasets)):
                datai = np.array(data_list[i])

                if i < 5:
                    print(f'{plot_attrs[i]['label']} (Normal):\n'
                     f'     2025-2050: {(datai[25] - datai[50]) / datai[0] *100:.2f} %;\n'
                     f'     2025-2100: {(datai[25] - datai[100]) / datai[0] *100:.2f} %')
                else:
                    print(f'{plot_attrs[i]['label']} (QDM):\n'
                     f'     2025-2050: {(datai[25] - datai[50]) / datai[0] *100:.2f} %;\n'
                     f'     2025-2100: {(datai[25] - datai[100]) / datai[0] *100:.2f} %')

                try:
                    critical_year = np.where(datai <= datai[0]*0.5)[0][0] + 2000
                    print(f'    Year with half {plot_type}: {critical_year}')
                except:
                    print(f'    Year with half {plot_type}: None')
                try:
                    critical_year = np.where(datai <= datai[0]*0.25)[0][0] + 2000
                    print(f'    Year with 25% {plot_type}: {critical_year}')
                except:
                    print(f'    Year with 25% {plot_type}: None')

        if display_legend:
            ax.legend(fontsize=18, ncol=2, 
                        bbox_to_anchor=(1.2, 1.2, 1.5, 2),
                        loc='lower right',
                        mode='expand')
        
        # 保存或显示
        if save_dir:
            if display_legend:
                save_path = os.path.join(save_dir, 'legend', f'{region_name}_legend.pdf')
            else:
                save_path = os.path.join(save_dir, f'{region_name}.pdf')

            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, dpi=600, bbox_inches='tight')
            plt.show()
        else:
            plt.show()
        
        # print(f'{region_name} processed!')

        for ds in datasets:
            if ds is not None:
                ds.close()
        for std in std_datasets:
            if std is not None:
                std.close()
                
    return None


In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj')

plot_future_projection_comparison([region_dirs[3]], plot_type='elevation', save_dir=save_dir, display_legend=True)

In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]

plot_future_projection_comparison([region_dirs[3]], plot_type='volume', save_dir=None, display_legend=True)

In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]

plot_future_projection_comparison([region_dirs[3]], plot_type='area', save_dir=None, display_legend=True)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj', 'area')
plot_future_projection_comparison(region_dirs, plot_type='area', save_dir=save_dir)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj', 'volume')
plot_future_projection_comparison(region_dirs, plot_type='volume', save_dir=save_dir)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj', 'elevation')
plot_future_projection_comparison(region_dirs, plot_type='elevation', save_dir=save_dir)

#### 8.1 环形图

##### by year

In [ ]:
def create_white_to_color_cmap(target_color, name='custom', N=20):
    if isinstance(target_color, str):
        target_rgb = mcolors.to_rgb(target_color)
    else:
        target_rgb = target_color[:3]

    colors_list = [(1, 1, 1), target_rgb] # from white fading to target color
    return mcolors.LinearSegmentedColormap.from_list(name, colors_list, N=N)

# plot_configs = {
#     'ssp126': create_white_to_color_cmap(plt.cm.Accent(0), 'ssp126_cmap'),
#     'ssp245': create_white_to_color_cmap(plt.cm.Accent(2), 'ssp245_cmap'),
#     'ssp370': create_white_to_color_cmap(plt.cm.Accent(6), 'ssp370_cmap'),
#     'ssp585': create_white_to_color_cmap('red', 'ssp585_cmap'),
# }
plot_configs = {
    'ssp126': create_white_to_color_cmap(plt.cm.Accent(0), 'ssp126_cmap'),
    'ssp245': plt.cm.Blues,
    'ssp370': create_white_to_color_cmap(plt.cm.Accent(6), 'ssp370_cmap'),
    'ssp585': plt.cm.Reds,
}

presented_scenarios = ['ssp245', 'ssp585']
presented_regions = region_dirs[3:4]
presented_files = [
    'run_output_cmip6_normal_spinup_all.nc', 
    'run_output_cmip6_repu_his_extremes_QDM_spinup_all.nc',
]
plot_var = 'volume'

for region_dir in presented_regions:
    presented_region_files = [os.path.join(region_dir, item) for item in presented_files]
    region_name = region_dir.split(os.sep)[-1]
    region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
    region_name_clean = region_name_clean.split(']')[1]

    fig, ax = plt.subplots(1, 1, figsize=(5, 4), dpi=600, subplot_kw=dict(projection='polar'))

    volume_matrix = []
    plot_matrix = []
    time_array = np.arange(2000, 2101)
    for ssp in presented_scenarios:
        for path in presented_region_files:
            volume_array = xr.open_dataset(path).sum(
                        dim='rgi_id', skipna=True, keep_attrs=True
                    ).sel(time=slice(2000, 2100), SSP=ssp)[plot_var]
            volume_matrix.append(volume_array.values) # 2000.1-2100.1
            plot_matrix.append(plot_configs[ssp]) # [normal, extreme, normal, extreme, ...]
    # volume percentage series
    volume_percent_matrix = [array/array[0]*100 for array in volume_matrix]
    
    n_intervals = len(time_array) - 25 #76, 2025-2100, for ring display
    theta = np.linspace(0, 2*np.pi, n_intervals, endpoint=False)
    width = 2 * np.pi / n_intervals
    norm = plt.Normalize(vmin=10, vmax=100)

    # radius
    r_inner_start = 0
    ring_height = 0.25
    r_outer = r_inner_start + len(volume_percent_matrix) * ring_height

    for ring_i in range(len(volume_percent_matrix)):
        r_inner = r_inner_start + ring_i * ring_height
        percent_array = volume_percent_matrix[ring_i][25:] # 2025-2100
        cmap = plot_matrix[ring_i]
        colors = cmap(norm(percent_array))

        bars = ax.bar(theta, ring_height, width=width, bottom=r_inner, 
                        color=colors, edgecolor='none', linewidth=0)
        
        idx_50 = np.where(percent_array <= 50)[0]
        if len(idx_50) > 0:
            ax.vlines(theta[idx_50[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='-', zorder=10)
        
        idx_25 = np.where(percent_array <= 25)[0]
        if len(idx_25) > 0:
            ax.vlines(theta[idx_25[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='--', zorder=10)
        
        if ring_i > 0:
            ax.plot(np.linspace(0, 2*np.pi, 100), [r_inner]*100, color='k', linewidth=1, zorder=10, alpha=0.7)

    ax.plot(np.linspace(0, 2*np.pi, 100), [r_outer]*100, color='k', linewidth=1.5, zorder=10)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1) # along clock
    ax.set_yticks([])
    ax.set_yticklabels([])

    year_labels = np.arange(2025, 2100, 15)
    label_angles = [theta[year-2025] for year in year_labels]
    ax.set_xticks(label_angles)
    ax.set_xticklabels(year_labels, fontsize=14)

    tick_space = 0.08 
    ax.set_rlim(0, r_outer + tick_space)
    ax.tick_params(axis='x', pad=8)
    ax.spines['polar'].set_visible(False)
    for angle in label_angles:
        ax.vlines(angle, r_outer, r_outer + tick_space, color='k', linewidth=1.5, zorder=10)

    custom_cmap = plt.cm.Greys #create_white_to_color_cmap('k', N=20)
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])
    cbar_ax = fig.add_axes(rect=[1.1, -0.2, 0.05, 1.4])  # [left, bottom, width, height]
    cbar = fig.colorbar(sm, cax=cbar_ax, shrink=0.6)
    cbar.ax.set_ylim([0, 100])
    cbar.ax.yaxis.set_major_locator(MultipleLocator(20))
    cbar.set_label('Volume percentage (%)', fontsize=14)

    # ax.set_title(region_name_clean, fontsize=14, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.show()

##### by percent

In [ ]:
def create_white_to_color_cmap(target_color, name='custom', N=20):
    if isinstance(target_color, str):
        target_rgb = mcolors.to_rgb(target_color)
    else:
        target_rgb = target_color[:3]

    colors_list = [(1, 1, 1), target_rgb] # from white fading to target color
    return mcolors.LinearSegmentedColormap.from_list(name, colors_list, N=N)

# plot_configs = {
#     'ssp126': create_white_to_color_cmap(plt.cm.Accent(0), 'ssp126_cmap'),
#     'ssp245': create_white_to_color_cmap(plt.cm.Accent(2), 'ssp245_cmap'),
#     'ssp370': create_white_to_color_cmap(plt.cm.Accent(6), 'ssp370_cmap'),
#     'ssp585': create_white_to_color_cmap('red', 'ssp585_cmap'),
# }
plot_configs = {
    'ssp126': create_white_to_color_cmap(plt.cm.Accent(0), 'ssp126_cmap'),
    'ssp245': plt.cm.Blues_r,
    'ssp370': create_white_to_color_cmap(plt.cm.Accent(6), 'ssp370_cmap'),
    'ssp585': plt.cm.Reds_r,
}

presented_scenarios = ['ssp245', 'ssp585']
presented_regions = region_dirs[3:4]
presented_files = [
    'run_output_cmip6_normal_spinup_all.nc', 
    'run_output_cmip6_repu_his_extremes_QDM_spinup_all.nc',
]
plot_var = 'volume'

for region_dir in presented_regions:
    presented_region_files = [os.path.join(region_dir, item) for item in presented_files]
    region_name = region_dir.split(os.sep)[-1]
    region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
    region_name_clean = region_name_clean.split(']')[1]

    fig, ax = plt.subplots(1, 1, figsize=(5, 4), dpi=600, subplot_kw=dict(projection='polar'))

    volume_matrix = []
    plot_matrix = []
    time_array = np.arange(2000, 2101)
    for ssp in presented_scenarios:
        for path in presented_region_files:
            volume_array = xr.open_dataset(path).sum(
                        dim='rgi_id', skipna=True, keep_attrs=True
                    ).sel(time=slice(2000, 2100), SSP=ssp)[plot_var]
            volume_matrix.append(volume_array.values) # 2000.1-2100.1
            plot_matrix.append(plot_configs[ssp]) # [normal, extreme, normal, extreme, ...]
    # volume percentage series
    volume_percent_matrix = [array/array[0]*100 for array in volume_matrix]
    percent_cuts = np.arange(90, -0.5, -2) # 90, 87.5, ..., 0

    n_intervals = len(percent_cuts)
    theta = np.linspace(0, 2*np.pi, n_intervals, endpoint=False)
    width = 2 * np.pi / n_intervals
    norm = plt.Normalize(vmin=2025, vmax=2100)

    # radius
    r_inner_start = 0
    ring_height = 0.25
    r_outer = r_inner_start + len(volume_percent_matrix) * ring_height

    for ring_i in range(len(volume_percent_matrix)):
        r_inner = r_inner_start + ring_i * ring_height
        percent_array = volume_percent_matrix[ring_i] # 2000-2100
        year_array = []
        for cut in percent_cuts:
            idx = np.where(percent_array <= cut)[0]
            if len(idx) > 0:
                year_array.append(idx[0]+2000)
            else:
                year_array.append(np.nan)

        year_array = np.array(year_array)
        cmap = plot_matrix[ring_i]
        colors = cmap(norm(year_array))
        colors = np.where(np.isnan(year_array)[:, np.newaxis], [1, 1, 1, 1], colors)

        bars = ax.bar(theta, ring_height, width=width, bottom=r_inner, 
                        color=colors, edgecolor='none', linewidth=0)
        
        idx_2050 = np.where(year_array >= 2050)[0]
        ax.vlines(theta[idx_2050[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='-', zorder=10)
        
        idx_2075 = np.where(year_array >= 2075)[0]
        ax.vlines(theta[idx_2075[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='--', zorder=10)
        
        if ring_i > 0:
            ax.plot(np.linspace(0, 2*np.pi, 100), [r_inner]*100, color='k', linewidth=1, zorder=10, alpha=0.7)

    ax.plot(np.linspace(0, 2*np.pi, 100), [r_outer]*100, color='k', linewidth=1.5, zorder=10)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1) # along clock
    ax.set_yticks([])
    ax.set_yticklabels([])

    xlabels = [f'-{(100-item):.0f} %' for item in percent_cuts[::10]] #-10%, 30, 50
    label_angles = theta[::10]
    ax.set_xticks(label_angles)
    ax.set_xticklabels(xlabels, fontsize=14)
    
    tick_space = 0.08 
    ax.set_rlim(0, r_outer + tick_space)
    ax.tick_params(axis='x', pad=8)
    ax.spines['polar'].set_visible(False)
    for angle in label_angles:
        ax.vlines(angle, r_outer, r_outer + tick_space, color='k', linewidth=1.5, zorder=10)

    custom_cmap = plt.cm.Greys_r #create_white_to_color_cmap('k', N=20)
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])
    cbar_ax = fig.add_axes(rect=[1.1, -0.2, 0.05, 1.4])  # [left, bottom, width, height]
    cbar = fig.colorbar(sm, cax=cbar_ax, shrink=0.6)
    cbar.ax.set_ylim([2025, 2100])
    cbar.ax.yaxis.set_major_locator(MultipleLocator(15))
    cbar.set_label('Year', fontsize=14)

    # ax.set_title(region_name_clean, fontsize=14, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.show()

##### Batch process

In [ ]:
def plot_percent_ring(region_dirs, plot_var='volume', presented_scenarios = ['ssp245', 'ssp585'],
                        save_dir=None, display_legend=False, display_ticklabel=False):
    plot_configs = {
        'ssp126': plt.cm.Greens_r,
        'ssp245': plt.cm.Blues_r,
        'ssp370': plt.cm.Oranges_r,
        'ssp585': plt.cm.Reds_r,
    }

    presented_files = [
        'run_output_cmip6_normal_spinup_all.nc', 
        'run_output_cmip6_repu_his_extremes_QDM_spinup_all.nc',
    ]

    for region_dir in region_dirs:
        presented_region_files = [os.path.join(region_dir, item) for item in presented_files]
        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[1]

        fig, ax = plt.subplots(1, 1, figsize=(5, 4), dpi=600, subplot_kw=dict(projection='polar'))

        volume_matrix = []
        plot_matrix = []
        time_array = np.arange(2000, 2101)
        for ssp in presented_scenarios:
            for path in presented_region_files:
                volume_array = xr.open_dataset(path).sum(
                            dim='rgi_id', skipna=True, keep_attrs=True
                        ).sel(time=slice(2000, 2100), SSP=ssp)[plot_var]
                volume_matrix.append(volume_array.values) # 2000.1-2100.1
                plot_matrix.append(plot_configs[ssp]) # [normal, extreme, normal, extreme, ...]
        # volume percentage series
        volume_percent_matrix = [array/array[0]*100 for array in volume_matrix]
        percent_cuts = np.arange(90, -0.5, -1) # 90, 88, ..., 0

        n_intervals = len(percent_cuts)
        theta = np.linspace(0, 2*np.pi, n_intervals, endpoint=False)
        width = 2 * np.pi / n_intervals
        norm = plt.Normalize(vmin=2025, vmax=2100)

        # radius
        r_inner_start = 0
        ring_height = 0.25
        r_outer = r_inner_start + len(volume_percent_matrix) * ring_height

        for ring_i in range(len(volume_percent_matrix)):
            r_inner = r_inner_start + ring_i * ring_height
            percent_array = volume_percent_matrix[ring_i] # 2000-2100
            year_array = []
            for cut in percent_cuts:
                idx = np.where(percent_array <= cut)[0]
                if len(idx) > 0:
                    year_array.append(idx[0]+2000)
                else:
                    year_array.append(np.nan)

            year_array = np.array(year_array)
            cmap = plot_matrix[ring_i]
            colors = cmap(norm(year_array))
            colors = np.where(np.isnan(year_array)[:, np.newaxis], [1, 1, 1, 1], colors)

            bars = ax.bar(theta, ring_height, width=width, bottom=r_inner, 
                            color=colors, edgecolor='none', linewidth=0)
            
            idx_2050 = np.where(year_array >= 2050)[0]
            ax.vlines(theta[idx_2050[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='-', zorder=10)
            
            idx_2075 = np.where(year_array >= 2075)[0]
            ax.vlines(theta[idx_2075[0]], r_inner, r_inner+ring_height, color='black', linewidth=1.5, linestyle='--', zorder=10)
            
            if ring_i > 0:
                ax.plot(np.linspace(0, 2*np.pi, 100), [r_inner]*100, color='k', linewidth=1, zorder=10, alpha=0.7)

        ax.plot(np.linspace(0, 2*np.pi, 100), [r_outer]*100, color='k', linewidth=1.5, zorder=10)
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1) # along clock
        ax.set_yticks([])
        ax.set_yticklabels([])

        xlabels = [f'-{(100-item):.0f} %' for item in percent_cuts[::20]] #-10%, 30, 50
        label_angles = theta[::20]
        ax.set_xticks(label_angles)
        if display_ticklabel:
            ax.set_xticklabels(xlabels, fontsize=14)
        else:
            ax.set_xticklabels([])
        
        tick_space = 0.08 
        ax.set_rlim(0, r_outer + tick_space)
        ax.tick_params(axis='x', pad=8)
        ax.spines['polar'].set_visible(False)
        for angle in label_angles:
            ax.vlines(angle, r_outer, r_outer + tick_space, color='k', linewidth=1.5, zorder=10)

        if display_legend:
            custom_cmap = plt.cm.Blues_r #create_white_to_color_cmap('k', N=20)
            sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
            sm.set_array([])
            cbar_ax = fig.add_axes(rect=[-0.1, -0.2, 1.2, 0.05])  # [left, bottom, width, height]
            cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal', extend='both')
            cbar.ax.set_xlim([2025, 2100])
            cbar.ax.xaxis.set_major_locator(MultipleLocator(15))
            cbar.set_label('Year', fontsize=14)

        # ax.set_title(region_name_clean, fontsize=14, fontweight='bold', pad=20)
        if save_dir:
            if display_legend:
                save_path = os.path.join(save_dir, 'legend', f'{region_name}_legend.pdf')
            else:
                save_path = os.path.join(save_dir, f'{region_name}.pdf')

            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.tight_layout()
            plt.savefig(save_path, dpi=600, bbox_inches='tight')
            plt.show()
        else:
            plt.tight_layout()
            plt.show()
        

In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj', 'ring_fig')

plot_percent_ring([region_dirs[3]], save_dir=save_dir, display_legend=True, display_ticklabel=True)

In [ ]:
base_dir = os.path.dirname(CONFIG['oggm_dir'])
region_dirs = [os.path.join(base_dir, item) for item in os.listdir(base_dir)]
save_dir = os.path.join(CONFIG['output_dir'], 'future_proj', 'ring_fig')

plot_percent_ring(region_dirs, save_dir=save_dir, display_legend=False, display_ticklabel=False)

##### Basemap

In [ ]:
# 地理边界数据路径配置
boundary_paths = {
    'studyregion': "E:/HMA/HMA_boundary/HMA_one.shp",  # 研究区总边界
    'subregion': "E:/HMA_subregion/regions_hma_v03_zheng/boundary_mountain_regions_hma_v3_zheng_20200601.shp",  # 子区域
    'glacier': "E:/HMA/Hugonnet/per_glacier/rgi_hma.shp"  # 冰川边界（可选）
}

# 加载边界数据
studyregion = gpd.read_file(boundary_paths['studyregion'])
subregion = gpd.read_file(boundary_paths['subregion'])
glacier = gpd.read_file(boundary_paths['glacier'])
glacier['geometry'] = glacier.geometry.simplify(tolerance=0.1, preserve_topology=True)

print("✓ 边界数据加载成功")
print(f"  研究区边界: {len(studyregion)} 个要素")
print(f"  子区域: {len(subregion)} 个")
print(f"  冰川: {len(glacier)} 个")

In [ ]:
def plot_study_region_boundaries(
    show_studyregion=False,
    show_subregion=True,
    show_glacier=False,
    show_rivers=True,
    show_lakes=True,
    studyregion_color='darkred',
    studyregion_linewidth=3,
    subregion_color='maroon',
    subregion_linewidth=2,
    glacier_color='dodgerblue',
    glacier_alpha=0.5,
    basemap='arcgis_terrain',
    figsize=(16, 12),
    dpi=600,
    title=None,
    save_path=None
):
    """
    绘制高亚洲研究区边界地图（仅边界，无数据可视化）
    
    参数:
        show_studyregion: bool, 是否显示研究区总边界
        show_subregion: bool, 是否显示子区域边界
        show_glacier: bool, 是否显示冰川边界
        show_rivers: bool, 是否显示河流
        show_lakes: bool, 是否显示湖泊
        studyregion_color: str, 研究区总边界颜色
        studyregion_linewidth: float, 研究区总边界线宽
        subregion_color: str, 子区域边界颜色
        subregion_linewidth: float, 子区域边界线宽
        glacier_color: str, 冰川颜色
        glacier_alpha: float, 冰川透明度
        basemap: str, 底图类型 ('arcgis_terrain', 'arcgis_imagery', 'none')
        figsize: tuple, 图形大小
        dpi: int, 图形分辨率
        title: str, 地图标题（可选）
        save_path: str, 保存路径（可选）
    
    返回:
        fig, ax: matplotlib图形对象
    """
    # 底图配置
    basemap_urls = {
        'arcgis_terrain': 'https://server.arcgisonline.com/arcgis/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}.jpg',
        'arcgis_imagery': 'https://server.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.jpg',
        'arcgis_shaded': 'https://server.arcgisonline.com/arcgis/rest/services/World_Shaded_Relief/MapServer/tile/{z}/{y}/{x}.jpg'
    }
    
    # 创建图形
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    
    # 设置地图范围（高亚洲区域）
    extent = [63.6, 108.3, 24.5, 47.5]
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # 添加底图
    if basemap != 'none' and basemap in basemap_urls:
        tiles = cimgt.GoogleTiles(url=basemap_urls[basemap])
        ax.add_image(tiles, 9)
    
    # 添加自然地理要素
    if show_rivers:
        ax.add_feature(cfeat.RIVERS.with_scale('10m'), 
                      linewidth=1.5, zorder=1, alpha=0.6, color='steelblue')
    if show_lakes:
        ax.add_feature(cfeat.LAKES.with_scale('10m'), 
                      zorder=1, alpha=0.6, facecolor='lightblue', edgecolor='steelblue')
    
    # 绘制冰川（如果需要，在边界之前绘制）
    if show_glacier:
        glacier.plot(ax=ax, color=glacier_color, edgecolor=glacier_color, 
                    linewidth=0.2, alpha=glacier_alpha, zorder=2)
    
    # 绘制子区域边界
    if show_subregion:
        subregion.plot(ax=ax, facecolor="none", edgecolor=subregion_color, 
                      linewidth=subregion_linewidth, zorder=5)
    
    # 绘制研究区总边界（在最上层）
    if show_studyregion:
        studyregion.plot(ax=ax, facecolor="none", edgecolor=studyregion_color, 
                        linewidth=studyregion_linewidth, zorder=6, linestyle='-')
    
    # 添加网格线
    #ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.3, color='gray', zorder=0)
    
    # 设置坐标轴
    # ax.set_xticks(np.arange(65, extent[1], 5), crs=ccrs.PlateCarree())
    # ax.set_yticks(np.arange(26, extent[3], 3), crs=ccrs.PlateCarree())
    # ax.xaxis.set_major_formatter(LongitudeFormatter())
    # ax.yaxis.set_major_formatter(LatitudeFormatter())
    # ax.tick_params(axis='both', labelsize=15, direction='out', pad=5)
    # ax.tick_params(axis='y', labelrotation=0)
    # ax.tick_params(axis='x', labelrotation=0)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off') 
    
    # 添加标题
    if title:
        ax.set_title(title, fontsize=22, pad=20, family='Arial', weight='bold')
    
    # 保存图形
    if save_path:
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
    
    return fig, ax


In [ ]:
fig, ax = plot_study_region_boundaries(
    show_studyregion=False,
    show_subregion=True,
    show_glacier=True,          # 显示冰川
    show_rivers=True,
    show_lakes=True,
    subregion_color='maroon',
    subregion_linewidth=2,
    glacier_color='dodgerblue', # 冰川用蓝色
    glacier_alpha=0.5,          # 半透明
    basemap='arcgis_terrain',
    figsize=(18, 14),
    dpi=600,
    save_path=os.path.join(CONFIG['output_dir'], 'future_proj', 'ring_fig', 'basemap', 'basemap.pdf')
)
plt.show()


## 9. 非线性关系分析（In[11]）

分析气候极端强度与冰川质量平衡之间的非线性关系，支持多项式拟合与上下界估计。


In [ ]:
def analyze_nonlinear_relationship(region_dirs, score, 
                                   poly_degree=2,  # 可配置多项式阶数
                                   factor=2,
                                   remove_last_n=0,  # 去除最后n个点
                                   display_legend=False,
                                   show_mean_poly=True,
                                   save_path=None):
    """
    分析气候极端强度与冰川质量平衡之间的非线性关系
    
    参数:
        region_dirs: list
        score_path: str, 极端强度数据Excel文件路径
        poly_degree: int, 多项式拟合阶数（建议2或3）
        factor: float, 标准差倍数（用于计算上下界）
        remove_last_n: int, 去除最后n个极端值点（避免不稳定）
        save_path: str, 保存路径（可选）
    """
    # 1. 加载数据
    smb_df = pd.DataFrame({}, columns=np.arange(2000, 2025))
    score_df = pd.DataFrame({}, columns=np.arange(2000, 2025))
    for region_dir in region_dirs:
        file_path = os.path.join(region_dir, 'run_output_hist_spinup_all.nc')
        ds = xr.open_dataset(file_path).sel(time=slice(2000,2025))
        ds_total = ds.sum(dim='rgi_id', skipna=True, keep_attrs=True)

        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[1]

        smb = calculate_smb_from_volume(ds_total)
        smb_df.loc[region_name_clean] = smb #每个子区域，mb from 2000 to 2024

        # aggregrate score to subregions
        rgi_ids = ds['rgi_id'].values
        selected_score = score.loc[rgi_ids]
        for year in range(2000, 2025):
            area_weight = ds.sel(time=year)['area'].values
            # score_df.loc[region_name_clean, year] = weight_average(selected_score[year].values, area_weight)#面积加权
            score_df.loc[region_name_clean, year] = selected_score[year].median()

        ds.close()
        print(f'{region_name_clean} done!')

    smb_df = smb_df.T # columns: regions
    score_df = score_df.T

    # 2. 合并数据
    n_regions = len(smb_df.columns)
    n_years = len(smb_df.index)
    region_names = smb_df.columns.tolist()
    data = np.zeros([n_regions * n_years, 2])
    
    for i in range(n_regions):
        for j in range(n_years):
            k = n_regions * i + j
            data[k][0] = score_df.loc[2000+j, region_names[i]]
            data[k][1] = smb_df.loc[2000+j, region_names[i]]
    
    # # 3. OLS线性回归（用于参考）
    # res = sm.OLS(data[:, 1], data[:, 0]).fit()
    # pred_ols = res.get_prediction()
    # iv_l = pred_ols.summary_frame()["obs_ci_lower"]
    # iv_u = pred_ols.summary_frame()["obs_ci_upper"]
    
    # 4. 计算唯一值和动态范围
    ind = np.sort(np.unique(data[:, 0]))
    imax = np.max(ind) + 1
    x_fit = np.arange(0, imax, 0.5) # np.max(ind) + 0.5
    
    # 5. 多项式拟合
    # 线性拟合
    p1 = np.polyfit(data[:, 0], data[:, 1], 1)
    y1 = np.polyval(p1, x_fit)
    
    # 多项式拟合（可配置阶数）
    p2 = np.polyfit(data[:, 0], data[:, 1], poly_degree)
    y2 = np.polyval(p2, x_fit)
    
    # 6. 计算每个极端强度值下的统计量
    mean = np.zeros(len(ind)) # 和ind对应
    std = np.zeros(len(ind))
    
    for i in range(len(ind)):
        mask = data[:, 0] == ind[i]
        mean[i] = np.mean(data[mask, 1])
        std[i] = np.std(data[mask, 1])
    
    # 7. 拟合上下界（基于统计）
    # 上界：mean + factor*std
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p3 = np.polyfit(ind[0:-remove_last_n], 
                        mean[0:-remove_last_n] + factor * std[0:-remove_last_n], 2)
    else:
        p3 = np.polyfit(ind, mean + factor * std, 2)
    y3 = np.polyval(p3, x_fit)
    
    # 下界：mean - factor*std（去除最后n个点，因为极端值可能不稳定）
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p4 = np.polyfit(ind[0:-remove_last_n], 
                       mean[0:-remove_last_n] - factor * std[0:-remove_last_n], 2)
    else:
        p4 = np.polyfit(ind, mean - factor * std, 2)
    y4 = np.polyval(p4, x_fit)
    
    # 均值线
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p5 = np.polyfit(ind[0:-remove_last_n], mean[0:-remove_last_n], 2)
    else:
        p5 = np.polyfit(ind, mean, 2)
    y5 = np.polyval(p5, x_fit)
    
    # 8. 计算相关系数（使用多项式拟合）
    cc = np.corrcoef(data[:, 0], data[:, 1])[0, 1]
    
    # 9. 绘图
    fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)
    
    # 绘制散点图
    for region_name in region_names:
        ax.scatter(score_df[region_name], smb_df[region_name], s=30)
    
    # 绘制拟合曲线,整体拟合
    ax.plot(x_fit, y1, linestyle='--', linewidth=5, label='Linear fitting', color='k')
    if show_mean_poly:
        ax.plot(x_fit, y5, linestyle='-', linewidth=5, label='Polynomial fitting', color='orange')
    
    # 绘制上下界（使用统计拟合的下界，而不是固定偏移）
    ax.plot(x_fit, y3, linestyle='-', linewidth=3, color='b', label='Upper and lower bound')
    ax.plot(x_fit, y4, linestyle='-', linewidth=3, color='b')
    
    # 添加相关系数文本
    ax.text(0.7, 0.80, f'$CC$ = {cc:.2f}',
           fontdict=dict(fontsize=18, family='Arial', color='k'), 
           transform=ax.transAxes)
    
    # 设置坐标轴
    ax.tick_params(axis='both', labelsize=16)
    ax.set_ylabel('Mass balance (m w.e./yr)', fontsize=18, labelpad=2)
    ax.set_xlabel('Extreme intensity ', 
                 fontsize=18, labelpad=2)
    # ax.xaxis.set_major_locator(MultipleLocator(base=1))
    ax.grid(axis='y', linestyle='--', alpha=0.5, linewidth=0.3)
    ax.grid(axis='x', linestyle='--', alpha=1, linewidth=0.8)
    
    if display_legend:
        ax.legend(fontsize=20, ncol=3, 
                    bbox_to_anchor=(1.2, 1.2, 3, 2),
                    loc='lower right',
                    mode='expand')
    
    # 保存或显示
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        plt.show()
    else:
        plt.show()
    
    # 返回结果
    results = {
        'data': data,
        'cc': cc,
        'unique_intensities': ind,
        'mean_by_intensity': mean,
        'std_by_intensity': std,
    }
    
    return results

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_median')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_all_legend.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_all, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=True,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_median')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_all.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_all, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_median')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_temp.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_temp, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=0,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_median')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_prcp.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_prcp, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

### 面积加权尝试

In [ ]:
def analyze_nonlinear_relationship(region_dirs, score, 
                                   poly_degree=2,  # 可配置多项式阶数
                                   factor=2,
                                   remove_last_n=0,  # 去除最后n个点
                                   display_legend=False,
                                   show_mean_poly=True,
                                   save_path=None):
    # 1. 加载数据
    smb_df = pd.DataFrame({}, columns=np.arange(2000, 2025))
    score_df = pd.DataFrame({}, columns=np.arange(2000, 2025))
    for region_dir in region_dirs:
        file_path = os.path.join(region_dir, 'run_output_hist_spinup_all.nc')
        ds = xr.open_dataset(file_path).sel(time=slice(2000,2025))
        ds_total = ds.sum(dim='rgi_id', skipna=True, keep_attrs=True)

        region_name = region_dir.split(os.sep)[-1]
        region_name_clean = region_name.replace('-','/') if '-' in region_name else region_name
        region_name_clean = region_name_clean.split(']')[1]

        smb = calculate_smb_from_volume(ds_total)
        smb_df.loc[region_name_clean] = smb #每个子区域，mb from 2000 to 2024

        # aggregrate score to subregions
        rgi_ids = ds['rgi_id'].values
        selected_score = score.loc[rgi_ids]
        for year in range(2000, 2025):
            area_weight = ds.sel(time=year)['area'].values
            score_df.loc[region_name_clean, year] = weight_average(selected_score[year].values, area_weight)#面积加权
            # score_df.loc[region_name_clean, year] = selected_score[year].median()

        ds.close()
        # print(f'{region_name_clean} done!')

    smb_df = smb_df.T # columns: regions
    score_df = score_df.T

    # 2. 合并数据
    n_regions = len(smb_df.columns)
    n_years = len(smb_df.index)
    region_names = smb_df.columns.tolist()
    data = np.zeros([n_regions * n_years, 2])
    
    for i in range(n_regions):
        for j in range(n_years):
            k = n_regions * i + j
            data[k][0] = score_df.loc[2000+j, region_names[i]]
            data[k][1] = smb_df.loc[2000+j, region_names[i]]
    
    # # 3. OLS线性回归（用于参考）
    # res = sm.OLS(data[:, 1], data[:, 0]).fit()
    # pred_ols = res.get_prediction()
    # iv_l = pred_ols.summary_frame()["obs_ci_lower"]
    # iv_u = pred_ols.summary_frame()["obs_ci_upper"]
    
    # 4. 计算范围
    imax = ceil(np.max(data[:, 0]))
    x_fit = np.arange(0, imax, 0.2) # np.max(ind) + 0.5
    
    # 5. 多项式拟合
    # 线性拟合
    p1 = np.polyfit(data[:, 0], data[:, 1], 1)
    y1 = np.polyval(p1, x_fit)
    
    # 多项式拟合（可配置阶数）
    p2 = np.polyfit(data[:, 0], data[:, 1], poly_degree)
    y2 = np.polyval(p2, x_fit)
    
    # 6. 计算每个极端强度值下的统计量
    mean = []
    std = []
    ind = []
    for i in range(imax):
        idx = (data[:, 0] >= i) & (data[:, 0] < i+1)
        if np.sum(idx) > 5:
            mean.append(np.mean(data[idx, 1]))
            std.append(np.std(data[idx, 1]))
            ind.append(np.mean(data[idx, 0]))

    mean = np.array(mean)
    std = np.array(std)
    ind = np.array(ind)
    # 7. 拟合上下界（基于统计）
    # 上界：mean + factor*std
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p3 = np.polyfit(ind[0:-remove_last_n], 
                        mean[0:-remove_last_n] + factor * std[0:-remove_last_n], 2)
    else:
        p3 = np.polyfit(ind, mean + factor * std, 2)
    y3 = np.polyval(p3, x_fit)
    
    # 下界：mean - factor*std（去除最后n个点，因为极端值可能不稳定）
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p4 = np.polyfit(ind[0:-remove_last_n], 
                       mean[0:-remove_last_n] - factor * std[0:-remove_last_n], 2)
    else:
        p4 = np.polyfit(ind, mean - factor * std, 2)
    y4 = np.polyval(p4, x_fit)
    
    # 均值线
    if remove_last_n > 0 and len(ind) > remove_last_n:
        p5 = np.polyfit(ind[0:-remove_last_n], mean[0:-remove_last_n], 2)
    else:
        p5 = np.polyfit(ind, mean, 2)
    y5 = np.polyval(p5, x_fit)
    
    # 8. 计算相关系数（使用多项式拟合）
    cc = np.corrcoef(data[:, 0], data[:, 1])[0, 1]
    
    # 9. 绘图
    fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)
    
    # 绘制散点图
    for region_name in region_names:
        ax.scatter(score_df[region_name], smb_df[region_name], s=30)
    
    # 绘制拟合曲线,整体拟合
    ax.plot(x_fit, y1, linestyle='--', linewidth=5, label='Linear fitting', color='k')
    if show_mean_poly:
        ax.plot(x_fit, y5, linestyle='-', linewidth=5, label='Polynomial fitting', color='orange')
    
    # 绘制上下界（使用统计拟合的下界，而不是固定偏移）
    ax.plot(x_fit, y3, linestyle='-', linewidth=3, color='b', label='Upper and lower bound')
    ax.plot(x_fit, y4, linestyle='-', linewidth=3, color='b')
    
    # 添加相关系数文本
    ax.text(0.7, 0.80, f'$CC$ = {cc:.2f}',
           fontdict=dict(fontsize=18, family='Arial', color='k'), 
           transform=ax.transAxes)
    
    # 设置坐标轴
    ax.tick_params(axis='both', labelsize=16)
    ax.set_ylabel('Mass balance (m w.e./yr)', fontsize=18, labelpad=2)
    ax.set_xlabel('Extreme intensity ', 
                 fontsize=18, labelpad=2)
    # ax.xaxis.set_major_locator(MultipleLocator(base=1))
    ax.grid(axis='y', linestyle='--', alpha=0.5, linewidth=0.3)
    ax.grid(axis='x', linestyle='--', alpha=1, linewidth=0.8)
    
    if display_legend:
        ax.legend(fontsize=20, ncol=3, 
                    bbox_to_anchor=(1.2, 1.2, 3, 2),
                    loc='lower right',
                    mode='expand')
    
    # 保存或显示
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        plt.show()
    else:
        plt.show()
    
    # 返回结果
    results = {
        'data': data,
        'cc': cc,
        'unique_intensities': ind,
        'mean_by_intensity': mean,
        'std_by_intensity': std,
    }
    
    return results

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_area_average')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_all.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_all, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_area_average')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_all_legend.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_all, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=True,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_area_average')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_temp.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_temp, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=1,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

In [ ]:
save_dir = os.path.join(CONFIG['output_dir'], 'nonlinear_area_average')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'nonlinear_prcp.pdf')
stat_summary = analyze_nonlinear_relationship(region_dirs, score_prcp, 
                                                poly_degree=2,
                                                factor=2,
                                                remove_last_n=2,
                                                display_legend=False,
                                                show_mean_poly=True,
                                                save_path=save_path)

## 10. ERA5 climate vs. CMIP6 climate

In [ ]:
plot_configs = {
    'ssp126': plt.cm.Accent(0),
    'ssp245': plt.cm.Accent(2),
    'ssp370': plt.cm.Accent(6), 
    'ssp585': 'red',
}
ssp_scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

fig, axes = plt.subplots(1, 2 , figsize=(14, 5))
axes = axes.flatten()
ax1 = axes[0]
ax2 = axes[1]

era_climate = xr.open_dataset(os.path.join(CONFIG['oggm_dir'], 'ERA5_HMA_climate_all.nc')).\
                sel(time=slice("1950", "2025")).mean(dim='rgi_id', skipna=True, keep_attrs=True)
cmip_climate = xr.open_dataset(os.path.join(CONFIG['oggm_dir'], 'CMIP6_HMA_climate_gcm_mean_all.nc')).\
                sel(time=slice("1950", "2101")).mean(dim='rgi_id', skipna=True, keep_attrs=True)
era_climate['time'] = pd.date_range(start="01/01/1950", end="12/01/2024", freq="MS")
cmip_climate['time'] = pd.date_range(start="01/01/1950", end="12/01/2100", freq="MS")

plot_date_range = pd.date_range(start="01/01/2000", end="12/01/2024", freq="MS")
era_climate_df = era_climate.to_dataframe().loc[plot_date_range] # df
era_temp = era_climate_df['temp'] # series
era_temp_positive = era_temp[era_temp>0]
annual_positive_degree_days = era_temp_positive.groupby(era_temp_positive.index.year).sum()
ax1.plot(annual_positive_degree_days.index, annual_positive_degree_days, color='k', linewidth=2, label='ERA5-Land', zorder=5)

era_prcp = era_climate_df['prcp'] # series
annual_prcp = era_prcp.groupby(era_prcp.index.year).sum()
ax2.plot(annual_prcp.index, annual_prcp, color='k', linewidth=2, zorder=5)

for ssp in ssp_scenarios:
    cmip_climate_df = cmip_climate.sel(SSP=ssp).to_dataframe().loc[plot_date_range]

    cmip_temp = cmip_climate_df['temp'] # series
    cmip_temp_positive = cmip_temp[cmip_temp>0]
    annual_positive_degree_days = cmip_temp_positive.groupby(cmip_temp_positive.index.year).sum()
    ax1.plot(annual_positive_degree_days.index, annual_positive_degree_days, color=plot_configs[ssp],
            linestyle='-', linewidth=1.5, label=f'CMIP6 {ssp.upper()}')

    cmip_prcp = cmip_climate_df['prcp'] # series
    annual_prcp = cmip_prcp.groupby(cmip_prcp.index.year).sum()
    ax2.plot(annual_prcp.index, annual_prcp, color=plot_configs[ssp],
            linestyle='-', linewidth=1.5)

ax1.set_ylabel('Annual PDD sum (°C)', fontsize=16)
ax2.set_ylabel('Annual precipitation (mm)', fontsize=16)
ax1.set_xlabel('Year', fontsize=16)
ax2.set_xlabel('Year', fontsize=16)

ax1.tick_params(labelsize=14)
ax1.yaxis.set_major_locator(MultipleLocator(2))
ax2.tick_params(labelsize=14)

ax1.legend(fontsize=16, ncol=3, 
        bbox_to_anchor=(0.3, -0.4, 1.7, 2),
        loc='lower right',
        mode='expand')

# plt.savefig(os.path.join(CONFIG['output_dir'], 'climate', 'ERAvsCMIP_hist.png'), dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# prcp trend using robust fitting
X = sm.add_constant(np.arange(len(era_prcp)))
y = era_prcp.values

rlm_model = RLM(y, X, M=sm.robust.norms.HuberT())
rlm_results = rlm_model.fit()

slope = rlm_results.params[1]
pvalue = rlm_results.pvalues[1]

print(f'slope = {slope} mm/month')
print(f'p = {pvalue}')

In [ ]:
# prcp trend using robust fitting
X = sm.add_constant(np.arange(len(annual_prcp)))[:-2]
y = annual_prcp.values[:-2]

rlm_model = RLM(y, X, M=sm.robust.norms.HuberT())
rlm_results = rlm_model.fit()

slope = rlm_results.params[1]
pvalue = rlm_results.pvalues[1]

print(f'slope = {slope} mm/year')
print(f'p = {pvalue}')

In [ ]:
# temp trend using robust fitting
X = sm.add_constant(np.arange(len(era_temp)))
y = era_temp.values

rlm_model = RLM(y, X, M=sm.robust.norms.HuberT())
rlm_results = rlm_model.fit()

slope = rlm_results.params[1]
pvalue = rlm_results.pvalues[1]

print(f'slope = {slope} ℃/month')
print(f'p = {pvalue}')

In [ ]:
# temp trend using robust fitting
X = sm.add_constant(np.arange(len(annual_positive_degree_days)))[:-2]
y = annual_positive_degree_days.values[:-2]

rlm_model = RLM(y, X, M=sm.robust.norms.HuberT())
rlm_results = rlm_model.fit()

slope = rlm_results.params[1]
pvalue = rlm_results.pvalues[1]

print(f'slope = {slope} ℃/year')
print(f'p = {pvalue}')